In [1]:
import requests
import pprint
from pprint import pformat
from bs4 import BeautifulSoup
import json
import pandas as pd
import logging
import matplotlib.pyplot as plt
import numpy as np
import math

In [2]:
# User agent for not being a robot
headers = {
            # 'User-Agent': '*',
            #
            #'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:65.0) Gecko/20100101 Firefox/65.0',
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36 OPR/64.0.3417.47',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
           'Accept-Language': 'fr-FR,en;q=0.5',
           'Accept-Encoding': 'gzip, deflate',
           'Connection': 'keep-alive',
            'Cache-Control': 'max-age=0',
           'Upgrade-Insecure-Requests': '1'}



In [3]:
logging.basicConfig(level=logging.CRITICAL, format='%(message)s')
logger = logging.getLogger()

In [4]:
class Scraper():
    def __init__(self, url, page_suffix):
        self.logger = logging.getLogger()
        self.headers = headers
        self.url = url
        self.page_suffix = page_suffix

    def reach_website(self, url, headers):
        # Create a session for the website
        s = requests.Session()
        s.headers.update(headers)
        print("URL : : : ", url )
        r = s.get(url)
        if r.status_code == 200:
            self.logger.info("WebSite reached")
            return r
        else:
            logging.error(f"WebSite unreachable, response : {r.text}")
            return None

    def find_scripts_on_page(self, response):
        print("response" , response)
        soup = BeautifulSoup(response.text, 'html.parser')
        self.logger.debug(f"Raw Soup {soup}")

        script_list = soup.find_all('script')
        self.logger.debug(f"Script list {script_list}")

        return script_list

    ## TO OVERLOAD ##
    def find_json_in_scripts(self, script_list):
        self.logger.critical("TO OVERLOAD")

        for script_item in script_list:
            if 'BALISE' in script_item.text:
                json_data = None  ## To replace

                return json_data

        logging.error("No BALISE on loaded Page")
        return None

    ## / TO OVERLOAD ##
    def get_appt_list(self, json_data):
        self.logger.critical("TO OVERLOAD")

        tmp_list_appt_1 = json_data['ads']
        tmp_list_appt_2 = json_data['ads_alu']
        tmp_list_appt = tmp_list_appt_1 + tmp_list_appt_2
        self.logger.debug(f"List appt : {pformat(tmp_list_appt)} ")

        # list_appt = list(filter(lambda x: x['publicationId'] is not None, list(tmp_list_appt)))

        return tmp_list_appt

    def show_data(self, data):
        df = pd.DataFrame(data)
        self.logger.debug(f" Data Frame : {df}")

        with pd.option_context('display.max_rows', None, 'display.max_columns',
                               None):  # more options can be specified also
            print(df)

    def get_appt_from_url(self, url):
        response = self.reach_website(url, headers)
        script_list = self.find_scripts_on_page(response)
        json_data = self.find_json_in_scripts(script_list)
        list_appt = self.get_appt_list(json_data)
        self.logger.info(f"Added {len(list_appt)} appartements.")

        return list_appt, json_data

    def get_full_list_appt(self):
        logger.info(f"Page scrapped : 1")
        list_appt, json_data = self.get_appt_from_url(self.url)

        logger.info(f"Total appartement scrapped {len(list_appt)} appartements.")
        #self.show_data(list_appt)
        return list_appt


In [5]:


class LebonCoin_Scraper(Scraper):

    def find_json_in_scripts(self, script_list):

        for script_item in script_list:
            if 'window.__REDIAL_PROPS__' in script_item.text:
                self.logger.debug(f"script_item window.__REDIAL_PROPS__ : {script_item.text} ")
                string_data = script_item.text.split('=', 1)[1]
                
                self.logger.debug(f"string_data : {pformat(string_data)} ")
                json_data = json.loads(string_data)[4]["data"]
                self.logger.debug(f"Raw JSON : {pformat(json_data)} ")

                return json_data

        logging.error("Non __REDIAL_PROPS__ on loaded Page")
        return None

    def get_appt_list(self, json_data):
        tmp_list_appt_1 = json_data['ads']
        tmp_list_appt_2 = json_data['ads_alu']
        tmp_list_appt = tmp_list_appt_1 + tmp_list_appt_2
        self.logger.debug(f"List appt : {pformat(tmp_list_appt)} ")

        # list_appt = list(filter(lambda x: x['publicationId'] is not None, list(tmp_list_appt)))

        return tmp_list_appt



In [6]:
print(json.dumps(headers, indent=4, sort_keys=True))

{
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "fr-FR,en;q=0.5",
    "Cache-Control": "max-age=0",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36 OPR/64.0.3417.47"
}


In [7]:
url_leboncoin = 'https://www.leboncoin.fr/_immobilier_/offres/ile_de_france/'
page_add_leboncoin = '&page=1'

In [8]:
scraper_1 = LebonCoin_Scraper(url_leboncoin, page_add_leboncoin)
#list_appt_1 = scraper_1.get_full_list_appt()

In [9]:
list_appt_1 = scraper_1.get_full_list_appt()

URL : : :  https://www.leboncoin.fr/_immobilier_/offres/ile_de_france/
response <Response [200]>


In [10]:
df = pd.DataFrame(list_appt_1)

In [11]:
df.columns

Index(['ad_type', 'attributes', 'body', 'category_id', 'category_name',
       'expiration_date', 'first_publication_date', 'has_phone', 'images',
       'index_date', 'list_id', 'location', 'options', 'owner', 'price',
       'price_calendar', 'status', 'subject', 'url'],
      dtype='object')

In [12]:
import datetime as dt

In [14]:
pd.to_datetime(df.index_date)[0]

Timestamp('2019-11-02 22:34:48')

In [26]:
df[["index_date","expiration_date", "first_publication_date"]]

,index_date,expiration_date,first_publication_date
0,2019-11-02 22:34:48,2020-01-01 22:34:48,2019-11-02 22:34:48
1,2019-11-02 22:33:15,2019-12-16 23:40:47,2019-10-17 23:40:47
2,2019-11-02 22:29:00,2020-01-01 22:29:00,2019-11-02 22:29:00
3,2019-11-02 22:28:47,2020-01-01 22:28:47,2019-11-02 22:28:47
4,2019-11-02 22:28:32,2020-01-01 21:18:34,2019-11-02 21:18:34
5,2019-11-02 22:27:44,2020-01-01 22:27:44,2019-11-02 22:27:44
6,2019-11-02 22:25:01,2020-01-01 16:02:17,2019-11-02 16:02:17
7,2019-11-02 22:23:18,2020-01-01 22:23:18,2019-11-02 22:23:18
8,2019-11-02 22:23:14,2020-01-01 22:23:14,2019-11-02 22:23:14
9,2019-11-02 22:21:25,2020-01-01 22:21:25,2019-11-02 22:21:25


In [27]:
#Get Location, body, 
def preprocessing_leboncoin(df = df):
    for loc, body, img, url, sub in zip(df.location, df.body, df.images, df.url, df.subject ) :
        print(url) 
        print(img) 
        print(loc) 
        print(body)
        print(sub)
        print("\n---------"*2)
    
    
preprocessing_leboncoin(df)

https://www.leboncoin.fr/locations/1700372349.htm
{'thumb_url': 'https://img1.leboncoin.fr/ad-thumb/0674ab933e07cb13b12137f5e061cac41523f922.jpg', 'small_url': 'https://img1.leboncoin.fr/ad-small/0674ab933e07cb13b12137f5e061cac41523f922.jpg', 'nb_images': 1, 'urls': ['https://img1.leboncoin.fr/ad-image/0674ab933e07cb13b12137f5e061cac41523f922.jpg'], 'urls_thumb': ['https://img1.leboncoin.fr/ad-thumb/0674ab933e07cb13b12137f5e061cac41523f922.jpg'], 'urls_large': ['https://img1.leboncoin.fr/ad-large/0674ab933e07cb13b12137f5e061cac41523f922.jpg']}
{'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '93', 'department_name': 'Seine-Saint-Denis', 'city_label': 'Drancy 93700', 'city': 'Drancy', 'zipcode': '93700', 'lat': 48.9234, 'lng': 2.44597, 'source': 'city', 'provider': 'here', 'is_shape': True}
1er contact par mail en précisant votre situation tél et mail
Grand appartement 3 pièces de 62 m² très agréable dans une belle résidence familiale, propre et sécurisée. L'appartem

In [16]:
set(df.location.apply( lambda x : str(sorted(list(x.keys())))))

{"['city', 'city_label', 'department_id', 'department_name', 'is_shape', 'lat', 'lng', 'provider', 'region_id', 'region_name', 'source', 'zipcode']"}

In [55]:
aaaa =  [{'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '93', 'department_name': 'Seine-Saint-Denis', 'city_label': 'Saint-Denis 93200', 'city': 'Saint-Denis', 'zipcode': '93200', 'lat': 48.93739, 'lng': 2.36482, 'source': 'city', 'provider': 'here', 'is_shape': True}, 'body': "Appartement à 8 minutes à pied de la station métro université M13 et de l'université Paris8\nIdéale pour jeunes actifs ou étudiants. Proche transport et commerces\nBien ensoleillé de 34m2 coté cour - rez de jardin, entrée , placard, grand sejour, cuisine,\nDouche / WC - grand jardin, chauffage et ballon d'eau chaude électrique\nDIsponible fin novembre 2019\nGarantie demandée et 2 mois de caution.\nLoyer mensuel 635 euros + 50 euros de charges\n\nPar email, merci de préciser votre situation et celle de votre garant", 'images': {'thumb_url': 'https://img2.leboncoin.fr/ad-thumb/54e4b38e100c338c8790f7b834d3032a9ade6a47.jpg', 'small_url': 'https://img2.leboncoin.fr/ad-small/54e4b38e100c338c8790f7b834d3032a9ade6a47.jpg', 'nb_images': 1, 'urls': ['https://img2.leboncoin.fr/ad-image/54e4b38e100c338c8790f7b834d3032a9ade6a47.jpg'], 'urls_thumb': ['https://img2.leboncoin.fr/ad-thumb/54e4b38e100c338c8790f7b834d3032a9ade6a47.jpg'], 'urls_large': ['https://img2.leboncoin.fr/ad-large/54e4b38e100c338c8790f7b834d3032a9ade6a47.jpg']}, 'url': 'https://www.leboncoin.fr/locations/1700397412.htm', 'subject': 'Appartement avec grand jardin', 'index_date': '2019-11-02 23:58:04', 'expiration_date': '2020-01-01 23:58:04', 'first_publication_date': '2019-11-02 23:58:04'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '75', 'department_name': 'Paris', 'city_label': 'Paris 75010', 'city': 'Paris', 'zipcode': '75010', 'lat': 48.87615, 'lng': 2.36233, 'source': 'city', 'provider': 'here', 'is_shape': True}, 'body': "un appartement de 72 m² au 1er étage avec ascenseur, immeuble pierre de taille, ravalée, digicode, interphone, 3 pièces autonome distribution en étoile, , meublés : entrée, cuisine séparée, avec grande fenêtre, équipée (table de cuisson vitrocéramique, réfrigérateur congélateur, four, micro ondes, lave linge), salle d'eau avec WC, chauffage individuel électrique, placards, parquet, moulures, balcon, bow- window ,traversant, double exposition, double vitrage, lumineux, calme. Idéal colocation, étudiant(e) s, stagiaires, jeunes actifs. Merci de nous contacter pour faire la réservation de la chambre. Veuillez nous laisser votre adresse e-mail.", 'images': {'thumb_url': 'https://img2.leboncoin.fr/ad-thumb/e3284066ada06f9dde9cdbc9db9cf53969844652.jpg', 'small_url': 'https://img2.leboncoin.fr/ad-small/e3284066ada06f9dde9cdbc9db9cf53969844652.jpg', 'nb_images': 3, 'urls': ['https://img2.leboncoin.fr/ad-image/e3284066ada06f9dde9cdbc9db9cf53969844652.jpg', 'https://img7.leboncoin.fr/ad-image/9252862ed39b46b3e1c8820597cd3e788db4ee1e.jpg', 'https://img7.leboncoin.fr/ad-image/59527907da19ae9306edf997fe2bf9cae6cfc831.jpg'], 'urls_thumb': ['https://img2.leboncoin.fr/ad-thumb/e3284066ada06f9dde9cdbc9db9cf53969844652.jpg', 'https://img7.leboncoin.fr/ad-thumb/9252862ed39b46b3e1c8820597cd3e788db4ee1e.jpg', 'https://img7.leboncoin.fr/ad-thumb/59527907da19ae9306edf997fe2bf9cae6cfc831.jpg'], 'urls_large': ['https://img2.leboncoin.fr/ad-large/e3284066ada06f9dde9cdbc9db9cf53969844652.jpg', 'https://img7.leboncoin.fr/ad-large/9252862ed39b46b3e1c8820597cd3e788db4ee1e.jpg', 'https://img7.leboncoin.fr/ad-large/59527907da19ae9306edf997fe2bf9cae6cfc831.jpg']}, 'url': 'https://www.leboncoin.fr/colocations/1700397365.htm', 'subject': 'Colocation a louer sur Paris 10 ème', 'index_date': '2019-11-02 23:57:45', 'expiration_date': '2020-01-01 23:57:45', 'first_publication_date': '2019-11-02 23:57:45'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '75', 'department_name': 'Paris', 'city_label': 'Paris 75010', 'city': 'Paris', 'zipcode': '75010', 'lat': 48.87615, 'lng': 2.36233, 'source': 'city', 'provider': 'lbc', 'is_shape': False}, 'body': "Dans le 10ème arrondissement de Paris, venez découvrir ce charmant appartement de 2 pièces entièrement rénové avec une courette privée de 5M² et une cave. \n\nIl se compose d'un salon, d'une cuisine équipée, d'une grande chambre de 15m² avec un grand dressing, d’une salle de douche et d’un WC séparé. Grande hauteur sous plafond de 2m85. Les fenêtres en double vitrage sont neuves.\n \nL'appartement est entièrement rénové et équipé : réfrigérateur /congélateur, four, induction, hotte, micro-onde, machine à laver ; un lit de 140x190 dans la chambre ; un canapé-lit de 140x190 (contient un matelas) dans le salon + un placard dans l'entrée, salle de bain avec carrelage au sol et au mur, une grande douche confortable de 80x120cm équipée d’une VMC. \nL'appartement se situe à proximité de la gare du nord, des lignes de métro M2, M7 et de nombreux bus. Montmartre et le Sacré Cœur sont à 10 minutes à pieds. \nCet appartement a été rénové avec grand soin. Nous cherchons un ou deux locataires particulièrement soigneux/soigneuses, car il est 100% neuf (première location après la rénovation). Pas d’animaux et idéal pour un jeune couple.\n \nLoyer : 1300€ + 100€ pour charges hors électricité \nL’appartement est disponible tout de suite.\n \nSélection sur dossier : contact par mail d’abord et visite après étude du dossier ;\nDépôt de garantie : 2 mois de loyer hors charges.\n\nListe des documents pour le dossier :\nLes trois derniers bulletins de salaire (3x le loyer demandé) ;\nLe contrat de travail ;\nLes 2 derniers avis d’imposition ;\nLes trois dernières quittances de loyer ;\nUne copie de sa pièce d’identité (carte d’identité, passeport…) ;\nPour les garants du locataire : une copie de la pièce d’identité, les trois derniers bulletins de salaire et l’avis d’imposition.\n\nAgences s’abstenir !", 'images': {'thumb_url': 'https://img3.leboncoin.fr/ad-thumb/77d3a290d6e4dc5365958872a8a9556a0f23511d.jpg', 'small_url': 'https://img3.leboncoin.fr/ad-small/77d3a290d6e4dc5365958872a8a9556a0f23511d.jpg', 'nb_images': 9, 'urls': ['https://img3.leboncoin.fr/ad-image/77d3a290d6e4dc5365958872a8a9556a0f23511d.jpg', 'https://img2.leboncoin.fr/ad-image/8871d6b245c7a76d4431360fcf4f332709197666.jpg', 'https://img1.leboncoin.fr/ad-image/1b2d79faf2eb2d83b1600f45ad5da1d1616f9dea.jpg', 'https://img6.leboncoin.fr/ad-image/bee385f52577cd70728f58a2d731c856b53c7b97.jpg', 'https://img1.leboncoin.fr/ad-image/819fb3d10e3e03d14170ca835cdcd796cf83d9cf.jpg', 'https://img2.leboncoin.fr/ad-image/62faef25cec90c44ddea179de7084b7a34b5a10a.jpg', 'https://img3.leboncoin.fr/ad-image/2b8f02338f5bd62544bd6ce33b6c2909018e9947.jpg', 'https://img2.leboncoin.fr/ad-image/9c6c1759777a823ffd425243cc2c2b64526b25f3.jpg', 'https://img0.leboncoin.fr/ad-image/b21168025a3c4b00d4842a0a70cebab4aa8cd379.jpg'], 'urls_thumb': ['https://img3.leboncoin.fr/ad-thumb/77d3a290d6e4dc5365958872a8a9556a0f23511d.jpg', 'https://img2.leboncoin.fr/ad-thumb/8871d6b245c7a76d4431360fcf4f332709197666.jpg', 'https://img1.leboncoin.fr/ad-thumb/1b2d79faf2eb2d83b1600f45ad5da1d1616f9dea.jpg', 'https://img6.leboncoin.fr/ad-thumb/bee385f52577cd70728f58a2d731c856b53c7b97.jpg', 'https://img1.leboncoin.fr/ad-thumb/819fb3d10e3e03d14170ca835cdcd796cf83d9cf.jpg', 'https://img2.leboncoin.fr/ad-thumb/62faef25cec90c44ddea179de7084b7a34b5a10a.jpg', 'https://img3.leboncoin.fr/ad-thumb/2b8f02338f5bd62544bd6ce33b6c2909018e9947.jpg', 'https://img2.leboncoin.fr/ad-thumb/9c6c1759777a823ffd425243cc2c2b64526b25f3.jpg', 'https://img0.leboncoin.fr/ad-thumb/b21168025a3c4b00d4842a0a70cebab4aa8cd379.jpg'], 'urls_large': ['https://img3.leboncoin.fr/ad-large/77d3a290d6e4dc5365958872a8a9556a0f23511d.jpg', 'https://img2.leboncoin.fr/ad-large/8871d6b245c7a76d4431360fcf4f332709197666.jpg', 'https://img1.leboncoin.fr/ad-large/1b2d79faf2eb2d83b1600f45ad5da1d1616f9dea.jpg', 'https://img6.leboncoin.fr/ad-large/bee385f52577cd70728f58a2d731c856b53c7b97.jpg', 'https://img1.leboncoin.fr/ad-large/819fb3d10e3e03d14170ca835cdcd796cf83d9cf.jpg', 'https://img2.leboncoin.fr/ad-large/62faef25cec90c44ddea179de7084b7a34b5a10a.jpg', 'https://img3.leboncoin.fr/ad-large/2b8f02338f5bd62544bd6ce33b6c2909018e9947.jpg', 'https://img2.leboncoin.fr/ad-large/9c6c1759777a823ffd425243cc2c2b64526b25f3.jpg', 'https://img0.leboncoin.fr/ad-large/b21168025a3c4b00d4842a0a70cebab4aa8cd379.jpg']}, 'url': 'https://www.leboncoin.fr/locations/1700389617.htm', 'subject': 'Appartement 2P 37m² rénové avec courette et cave', 'index_date': '2019-11-02 23:57:11', 'expiration_date': '2020-01-01 23:57:11', 'first_publication_date': '2019-11-02 23:57:11'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '78', 'department_name': 'Yvelines', 'city_label': 'Versailles 78000', 'city': 'Versailles', 'zipcode': '78000', 'lat': 48.80195, 'lng': 2.13109, 'source': 'city', 'provider': 'here', 'is_shape': True}, 'body': "Precisez nous votre mail pour plus d'informations sur la location\n\nCet appartement de 97m2 au 1er étage avec ascenseur. Immeuble récent offrant des prestations haut de gamme. Le bien traversant Est/Ouest se compose d'un hall d'entrée, d'une cuisine ouverte sur un grand séjour donnant sur une grande terrasse de 20 m². Une suite parentale avec une douche à l'italienne, deux autres chambres et une salle de bain. Système de chauffage en climatisation réversible, volets roulants centralisés. Appartement entièrement rénové en 2018. Parking.", 'images': {'thumb_url': 'https://img4.leboncoin.fr/ad-thumb/ed5a6c5b1b6acb821ae6daa29c946e14a22faddd.jpg', 'small_url': 'https://img4.leboncoin.fr/ad-small/ed5a6c5b1b6acb821ae6daa29c946e14a22faddd.jpg', 'nb_images': 3, 'urls': ['https://img4.leboncoin.fr/ad-image/ed5a6c5b1b6acb821ae6daa29c946e14a22faddd.jpg', 'https://img0.leboncoin.fr/ad-image/abcc1f5763858ffba9650f8ff3c5ec83953db8a0.jpg', 'https://img7.leboncoin.fr/ad-image/b235304b5df556a8eed7f9c6e9703c8a924e44a9.jpg'], 'urls_thumb': ['https://img4.leboncoin.fr/ad-thumb/ed5a6c5b1b6acb821ae6daa29c946e14a22faddd.jpg', 'https://img0.leboncoin.fr/ad-thumb/abcc1f5763858ffba9650f8ff3c5ec83953db8a0.jpg', 'https://img7.leboncoin.fr/ad-thumb/b235304b5df556a8eed7f9c6e9703c8a924e44a9.jpg'], 'urls_large': ['https://img4.leboncoin.fr/ad-large/ed5a6c5b1b6acb821ae6daa29c946e14a22faddd.jpg', 'https://img0.leboncoin.fr/ad-large/abcc1f5763858ffba9650f8ff3c5ec83953db8a0.jpg', 'https://img7.leboncoin.fr/ad-large/b235304b5df556a8eed7f9c6e9703c8a924e44a9.jpg']}, 'url': 'https://www.leboncoin.fr/locations/1700397270.htm', 'subject': 'Versailles T4', 'index_date': '2019-11-02 23:56:58', 'expiration_date': '2020-01-01 23:56:58', 'first_publication_date': '2019-11-02 23:56:58'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '78', 'department_name': 'Yvelines', 'city_label': 'Ecquevilly 78920', 'city': 'Ecquevilly', 'zipcode': '78920', 'lat': 48.9499, 'lng': 1.91856, 'source': 'city', 'provider': 'here', 'is_shape': True}, 'body': 'Au cœur du vieux Ecquevilly, centre ville, quartier calme.\nDans une copropriété, au 1er étage, appartement 2 pièces avec du cachet (46m2 -loi carrez, 52m2 surface utile).\n\nEntrée sur séjour avec poil à bois, cuisine ouverte et équipée, salon/bureau + mezzanine (chambre), salle de bain avec douche italienne, WC.\nChauffage électrique radiant + chauffage au bois.\n\n1 garage de 12 m2 servant de stockage dans la cour commune.\nStationnement gratuit dans la rue ou possibilité de garer sa voiture dans la cour commune.\n\nCharge de copropriété : 0€\nTaxe foncière 350€\n\nA proximité des commodités (crèches, établissements scolaires, parcs, commerce alimentaire, boulangeries, banque etc…).\nA 150m le bus 13 Gare des Mureaux en 10/15 min.\nGare des Mureaux 10 min en voiture (sûr) accès ligne J vers Paris St Lazare (35 min). Stationnement gratuit à 5 min à pied de la gare.\nAccès autoroute A13 / A14 à moins de 5 minutes en voiture.\nParis porte d’Auteuil à 25 min en voiture.\n\nDPE et GES en cours.\nIdéal primo accédant.\n\n1er contact par mail\n\nAGENCES ET INDEPENDANTS S’ABSTENIR SVP', 'images': {'thumb_url': 'https://img3.leboncoin.fr/ad-thumb/35d85dd92ac7e3f11bece9328677ce0ac2d688ac.jpg', 'small_url': 'https://img3.leboncoin.fr/ad-small/35d85dd92ac7e3f11bece9328677ce0ac2d688ac.jpg', 'nb_images': 3, 'urls': ['https://img3.leboncoin.fr/ad-image/35d85dd92ac7e3f11bece9328677ce0ac2d688ac.jpg', 'https://img7.leboncoin.fr/ad-image/0d425fa3ca1537900c849a4b66303248085cc17c.jpg', 'https://img5.leboncoin.fr/ad-image/8ddb7088973a5a6640d46c176d22e5338937b7df.jpg'], 'urls_thumb': ['https://img3.leboncoin.fr/ad-thumb/35d85dd92ac7e3f11bece9328677ce0ac2d688ac.jpg', 'https://img7.leboncoin.fr/ad-thumb/0d425fa3ca1537900c849a4b66303248085cc17c.jpg', 'https://img5.leboncoin.fr/ad-thumb/8ddb7088973a5a6640d46c176d22e5338937b7df.jpg'], 'urls_large': ['https://img3.leboncoin.fr/ad-large/35d85dd92ac7e3f11bece9328677ce0ac2d688ac.jpg', 'https://img7.leboncoin.fr/ad-large/0d425fa3ca1537900c849a4b66303248085cc17c.jpg', 'https://img5.leboncoin.fr/ad-large/8ddb7088973a5a6640d46c176d22e5338937b7df.jpg']}, 'url': 'https://www.leboncoin.fr/ventes_immobilieres/1700397203.htm', 'subject': 'Appartement dans une copropriété', 'index_date': '2019-11-02 23:56:47', 'expiration_date': '2020-01-01 23:56:47', 'first_publication_date': '2019-11-02 23:56:47'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '93', 'department_name': 'Seine-Saint-Denis', 'city_label': 'Montreuil 93100', 'city': 'Montreuil', 'zipcode': '93100', 'lat': 48.86453, 'lng': 2.44266, 'source': 'city', 'provider': 'here', 'is_shape': True}, 'body': "Precisez nous votre mail pour plus d'informations sur la location\n\nDans un bel immeuble ce magnifique appartement de 4 pièces au 1er étage avec ascenseur. L'appartement se compose d'une grande entrée, un séjour/salle à manger ouvert sur une cuisine entièrement aménagée et équipée, 3 chambres avec chacune sa salle de bains, 3 dressings, une buanderie et des toilettes invités. parking, 1 caves complètent ce bien d'exception.", 'images': {'thumb_url': 'https://img4.leboncoin.fr/ad-thumb/681a0dff466eb920a031b5adb4c4427f61f743f2.jpg', 'small_url': 'https://img4.leboncoin.fr/ad-small/681a0dff466eb920a031b5adb4c4427f61f743f2.jpg', 'nb_images': 3, 'urls': ['https://img4.leboncoin.fr/ad-image/681a0dff466eb920a031b5adb4c4427f61f743f2.jpg', 'https://img4.leboncoin.fr/ad-image/1c47961b544192666bf50035649f03b024330bee.jpg', 'https://img3.leboncoin.fr/ad-image/53278713b50abf8aa33195a07f7ef715d757a730.jpg'], 'urls_thumb': ['https://img4.leboncoin.fr/ad-thumb/681a0dff466eb920a031b5adb4c4427f61f743f2.jpg', 'https://img4.leboncoin.fr/ad-thumb/1c47961b544192666bf50035649f03b024330bee.jpg', 'https://img3.leboncoin.fr/ad-thumb/53278713b50abf8aa33195a07f7ef715d757a730.jpg'], 'urls_large': ['https://img4.leboncoin.fr/ad-large/681a0dff466eb920a031b5adb4c4427f61f743f2.jpg', 'https://img4.leboncoin.fr/ad-large/1c47961b544192666bf50035649f03b024330bee.jpg', 'https://img3.leboncoin.fr/ad-large/53278713b50abf8aa33195a07f7ef715d757a730.jpg']}, 'url': 'https://www.leboncoin.fr/locations/1700397139.htm', 'subject': 'Montreuil T4', 'index_date': '2019-11-02 23:56:12', 'expiration_date': '2020-01-01 23:56:12', 'first_publication_date': '2019-11-02 23:56:12'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '75', 'department_name': 'Paris', 'city_label': 'Paris 75011', 'city': 'Paris', 'zipcode': '75011', 'lat': 48.85765, 'lng': 2.38003, 'source': 'address', 'provider': 'here', 'is_shape': False}, 'body': 'KITCHENETTE/FOUR/MICRO ONDE/CAFETIERE/FRIGO/CONGELATEUR/LAVE VAISELLE/MACHINE A LAVER LINGE.\nTOILETTE INDEPENDANTE\nSDB AVEC CABINE DOUCHE\nTERRASSE\nPETITE VUE MONT BLANC.\nPARKING COUVERT (1 VEHICULE)\nPEUT CONVENIR A COUPLE (PERSONNES SERIEUSES)', 'images': {'thumb_url': 'https://img2.leboncoin.fr/ad-thumb/54c83bf6b7123e006edc79f429cd3b6494699258.jpg', 'small_url': 'https://img2.leboncoin.fr/ad-small/54c83bf6b7123e006edc79f429cd3b6494699258.jpg', 'nb_images': 2, 'urls': ['https://img2.leboncoin.fr/ad-image/54c83bf6b7123e006edc79f429cd3b6494699258.jpg', 'https://img7.leboncoin.fr/ad-image/3cc8d6181bdaa4b626266f931087fd3c27ff45c6.jpg'], 'urls_thumb': ['https://img2.leboncoin.fr/ad-thumb/54c83bf6b7123e006edc79f429cd3b6494699258.jpg', 'https://img7.leboncoin.fr/ad-thumb/3cc8d6181bdaa4b626266f931087fd3c27ff45c6.jpg'], 'urls_large': ['https://img2.leboncoin.fr/ad-large/54c83bf6b7123e006edc79f429cd3b6494699258.jpg', 'https://img7.leboncoin.fr/ad-large/3cc8d6181bdaa4b626266f931087fd3c27ff45c6.jpg']}, 'url': 'https://www.leboncoin.fr/locations/1700396862.htm', 'subject': 'Appartement 2 pièces 47 m²', 'index_date': '2019-11-02 23:54:20', 'expiration_date': '2020-01-01 23:54:20', 'first_publication_date': '2019-11-02 23:54:20'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '75', 'department_name': 'Paris', 'city_label': 'Paris 75009', 'city': 'Paris', 'zipcode': '75009', 'lat': 48.872282, 'lng': 2.34325, 'source': 'device', 'provider': 'android', 'is_shape': False}, 'body': 'Bonjour,\n\nDans un appartement calme et cosy, je loue un lit (partie supérieure d\'un lit superposé) que vous allez partager avec moi même dans un studio très très bien situé dans le 9eme à Paris et très bien aménagé, ligne 9 et 8 au pieds de l\'immeuble avec des musées, théâtres, le GRAND REX, restaurants, magasins, Auber à 10min à pieds ou 5" en transports en commun, gare du Nord et Châtelet à 15min, le quartier est très branché et animé,\n\nVous aurez à votre disposition les draps, un four à micro-ondes, les ustensiles de cuisine, un frigo et du rangements, \n\nA savoir, je ne dispose pas d\'une machine à laver et la wifi (une laverie à 4 minutes),\n\nIdéalement pour une personne de passage à Paris et qui cherche du calme et la sécurité, \n\nJe suis une étudiante discrète et souvent à l\'extérieur, c\'est exclusivement pour femmes sérieuses, propres et discrètes, idéalement pour étudiante ou jeune travailleuse,\n\nLe sérieux est un pilier pour moi, je vous pris de bien vouloir me contacter que si vous êtes réellement intéressées, \n\nAttention : la location est à partir du mois de Décembre, premier arrivé, premier servi, \n\nN\'hésitez pas à me contacter si besoin, je ferai un plaisir de vous répondre\n\nPS: le prix moyen d\'une nuit dans les hôtels dans le même lieu géographique est de 150Euro(s)/nuit :) ainsi, pour toute personne qui réserve 7 nuits, les suivantes sont à -20% \n\nÀ très vite,', 'images': {'nb_images': 0}, 'url': 'https://www.leboncoin.fr/locations/1700396465.htm', 'subject': 'Location courte durée', 'index_date': '2019-11-02 23:51:20', 'expiration_date': '2020-01-01 23:51:20', 'first_publication_date': '2019-11-02 23:51:20'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '78', 'department_name': 'Yvelines', 'city_label': 'Maurepas 78310', 'city': 'Maurepas', 'zipcode': '78310', 'lat': 48.76175, 'lng': 1.94486, 'source': 'city', 'provider': 'here', 'is_shape': True}, 'body': 'Proche centre ville et gare, appartement de 4 pièces de 76m2, lumineux et en parfait état.\n\nEntièrement parqueté, cet appartement de qualité offre : une grande entrée, un séjour avec balcon donnant sur rue pietonne, une cuisine aménagée et équipée, SDB et 3 chambres dont 2 avec placard, wc séparé. \nUne grande cave et une place de parking en sous-sol complètent ce bien.\n\nCentre médical, pharmacie, commerces et 5 lignes de bus à proximité. Ecole/ collège/ lycée à quelques min à pied. Acces gare de La Verriere et N10 rapide. Fibre optique. Aucun travaux à prévoir.', 'images': {'thumb_url': 'https://img1.leboncoin.fr/ad-thumb/5e1385ee90d21f1df56b7846bfecfb51bded7442.jpg', 'small_url': 'https://img1.leboncoin.fr/ad-small/5e1385ee90d21f1df56b7846bfecfb51bded7442.jpg', 'nb_images': 3, 'urls': ['https://img1.leboncoin.fr/ad-image/5e1385ee90d21f1df56b7846bfecfb51bded7442.jpg', 'https://img7.leboncoin.fr/ad-image/6af5c5a68840764d5c3390935e930e9c3c680740.jpg', 'https://img5.leboncoin.fr/ad-image/929688e0af09087a2ac74b03dc05cd05705fb837.jpg'], 'urls_thumb': ['https://img1.leboncoin.fr/ad-thumb/5e1385ee90d21f1df56b7846bfecfb51bded7442.jpg', 'https://img7.leboncoin.fr/ad-thumb/6af5c5a68840764d5c3390935e930e9c3c680740.jpg', 'https://img5.leboncoin.fr/ad-thumb/929688e0af09087a2ac74b03dc05cd05705fb837.jpg'], 'urls_large': ['https://img1.leboncoin.fr/ad-large/5e1385ee90d21f1df56b7846bfecfb51bded7442.jpg', 'https://img7.leboncoin.fr/ad-large/6af5c5a68840764d5c3390935e930e9c3c680740.jpg', 'https://img5.leboncoin.fr/ad-large/929688e0af09087a2ac74b03dc05cd05705fb837.jpg']}, 'url': 'https://www.leboncoin.fr/ventes_immobilieres/1700396114.htm', 'subject': 'Appartement F4 76m2', 'index_date': '2019-11-02 23:49:05', 'expiration_date': '2020-01-01 23:49:05', 'first_publication_date': '2019-11-02 23:49:05'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '75', 'department_name': 'Paris', 'city_label': 'Paris 75015', 'city': 'Paris', 'zipcode': '75015', 'lat': 48.84991, 'lng': 2.29288, 'source': 'address', 'provider': 'here', 'is_shape': False}, 'body': 'Petit studio meublé 10m². Près de la Tour Eiffel. Métro Ligne 6 Dupleix à 2min, Très proche des transports et des commerces\n2ème étage bien entretenue. vu sur jardin intérieur, calme. \nSéjour muni d’un lit une place et de rangement, avec un coin cuisine équipée.\nLa salle de bain est indépendante avec baignoire/lavabo et toilette séparée dans l’appartement.\n\nEau chaude et chauffage collectif.\nGardien, digicode.\n\nLoyer :525€ + charges 95 €\n\nLibre tout de suite.\n\nGaranties demandées. Si vous êtes intéressé. Dossier complet demandé lors de la visite\n\nContact le propriétaire : 07 62 05 40 71', 'images': {'thumb_url': 'https://img7.leboncoin.fr/ad-thumb/810eebe50965b013dff82ee5e4b032262f481ae1.jpg', 'small_url': 'https://img7.leboncoin.fr/ad-small/810eebe50965b013dff82ee5e4b032262f481ae1.jpg', 'nb_images': 3, 'urls': ['https://img7.leboncoin.fr/ad-image/810eebe50965b013dff82ee5e4b032262f481ae1.jpg', 'https://img2.leboncoin.fr/ad-image/f8e0e5c92553b89e09b47accad0e89cdec97cf0b.jpg', 'https://img7.leboncoin.fr/ad-image/21a7d04a51b49279d9536987b1f565548c2c3787.jpg'], 'urls_thumb': ['https://img7.leboncoin.fr/ad-thumb/810eebe50965b013dff82ee5e4b032262f481ae1.jpg', 'https://img2.leboncoin.fr/ad-thumb/f8e0e5c92553b89e09b47accad0e89cdec97cf0b.jpg', 'https://img7.leboncoin.fr/ad-thumb/21a7d04a51b49279d9536987b1f565548c2c3787.jpg'], 'urls_large': ['https://img7.leboncoin.fr/ad-large/810eebe50965b013dff82ee5e4b032262f481ae1.jpg', 'https://img2.leboncoin.fr/ad-large/f8e0e5c92553b89e09b47accad0e89cdec97cf0b.jpg', 'https://img7.leboncoin.fr/ad-large/21a7d04a51b49279d9536987b1f565548c2c3787.jpg']}, 'url': 'https://www.leboncoin.fr/locations/1700395985.htm', 'subject': 'Studio meublé près de la Tour Eiffel Métro Dupleix à 2min', 'index_date': '2019-11-02 23:48:13', 'expiration_date': '2020-01-01 23:48:13', 'first_publication_date': '2019-11-02 23:48:13'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '75', 'department_name': 'Paris', 'city_label': 'Paris 75006', 'city': 'Paris', 'zipcode': '75006', 'lat': 48.85089, 'lng': 2.33285, 'source': 'address', 'provider': 'here', 'is_shape': False}, 'body': 'KITCHENETTE/FOUR/MICRO ONDE/CAFETIERE/FRIGO/CONGELATEUR/LAVE VAISELLE/MACHINE A LAVER LINGE.\nTOILETTE INDEPENDANTE\nSDB AVEC CABINE DOUCHE\nTERRASSE\nPETITE VUE MONT BLANC.\nPARKING COUVERT (1 VEHICULE)\nPEUT CONVENIR A COUPLE (PERSONNES SERIEUSES)', 'images': {'thumb_url': 'https://img0.leboncoin.fr/ad-thumb/34d28883818848a5bd8dd042e73b06fa8c5bd717.jpg', 'small_url': 'https://img0.leboncoin.fr/ad-small/34d28883818848a5bd8dd042e73b06fa8c5bd717.jpg', 'nb_images': 3, 'urls': ['https://img0.leboncoin.fr/ad-image/34d28883818848a5bd8dd042e73b06fa8c5bd717.jpg', 'https://img0.leboncoin.fr/ad-image/fe555038386bf6945fe1a4829473ecace8adb81f.jpg', 'https://img3.leboncoin.fr/ad-image/9a6b6c7cd5079ea77c0ee091a3cb986141f9af91.jpg'], 'urls_thumb': ['https://img0.leboncoin.fr/ad-thumb/34d28883818848a5bd8dd042e73b06fa8c5bd717.jpg', 'https://img0.leboncoin.fr/ad-thumb/fe555038386bf6945fe1a4829473ecace8adb81f.jpg', 'https://img3.leboncoin.fr/ad-thumb/9a6b6c7cd5079ea77c0ee091a3cb986141f9af91.jpg'], 'urls_large': ['https://img0.leboncoin.fr/ad-large/34d28883818848a5bd8dd042e73b06fa8c5bd717.jpg', 'https://img0.leboncoin.fr/ad-large/fe555038386bf6945fe1a4829473ecace8adb81f.jpg', 'https://img3.leboncoin.fr/ad-large/9a6b6c7cd5079ea77c0ee091a3cb986141f9af91.jpg']}, 'url': 'https://www.leboncoin.fr/locations/1700395927.htm', 'subject': 'Studio cabine', 'index_date': '2019-11-02 23:47:49', 'expiration_date': '2020-01-01 23:47:49', 'first_publication_date': '2019-11-02 23:47:49'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '95', 'department_name': "Val-d'Oise", 'city_label': 'Argenteuil 95100', 'city': 'Argenteuil', 'zipcode': '95100', 'lat': 48.94209, 'lng': 2.24506, 'source': 'city', 'provider': 'here', 'is_shape': True}, 'body': 'KITCHENETTE/FOUR/MICRO ONDE/CAFETIERE/FRIGO/CONGELATEUR/LAVE VAISELLE/MACHINE A LAVER LINGE.\nTOILETTE INDEPENDANTE\nSDB AVEC CABINE DOUCHE\nTERRASSE\nPETITE VUE MONT BLANC.\nPARKING COUVERT (1 VEHICULE)\nPEUT CONVENIR A COUPLE (PERSONNES SERIEUSES)', 'images': {'thumb_url': 'https://img0.leboncoin.fr/ad-thumb/f28c7e8b29c1c02a6efe2573cc582ef104e8388b.jpg', 'small_url': 'https://img0.leboncoin.fr/ad-small/f28c7e8b29c1c02a6efe2573cc582ef104e8388b.jpg', 'nb_images': 2, 'urls': ['https://img0.leboncoin.fr/ad-image/f28c7e8b29c1c02a6efe2573cc582ef104e8388b.jpg', 'https://img3.leboncoin.fr/ad-image/430871f12e8682aff716c2e7fd58dcc85603ad94.jpg'], 'urls_thumb': ['https://img0.leboncoin.fr/ad-thumb/f28c7e8b29c1c02a6efe2573cc582ef104e8388b.jpg', 'https://img3.leboncoin.fr/ad-thumb/430871f12e8682aff716c2e7fd58dcc85603ad94.jpg'], 'urls_large': ['https://img0.leboncoin.fr/ad-large/f28c7e8b29c1c02a6efe2573cc582ef104e8388b.jpg', 'https://img3.leboncoin.fr/ad-large/430871f12e8682aff716c2e7fd58dcc85603ad94.jpg']}, 'url': 'https://www.leboncoin.fr/locations/1700395808.htm', 'subject': 'Appartement 2 pièces 49 m²', 'index_date': '2019-11-02 23:47:03', 'expiration_date': '2020-01-01 23:47:03', 'first_publication_date': '2019-11-02 23:47:03'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '92', 'department_name': 'Hauts-de-Seine', 'city_label': 'Vanves 92170', 'city': 'Vanves', 'zipcode': '92170', 'lat': 48.82118, 'lng': 2.29118, 'source': 'city', 'provider': 'here', 'is_shape': True}, 'body': "Colocation d'une chambre dans 92 contactée moi mp pour pluse De détaille", 'images': {'thumb_url': 'https://img1.leboncoin.fr/ad-thumb/6165629f3db6dd424f4cdc4c24f4c748d3c74dd1.jpg', 'small_url': 'https://img1.leboncoin.fr/ad-small/6165629f3db6dd424f4cdc4c24f4c748d3c74dd1.jpg', 'nb_images': 3, 'urls': ['https://img1.leboncoin.fr/ad-image/6165629f3db6dd424f4cdc4c24f4c748d3c74dd1.jpg', 'https://img0.leboncoin.fr/ad-image/982d50d10ae7b505e95c0b57a3b81b50b87d1ac8.jpg', 'https://img1.leboncoin.fr/ad-image/cbb97cd70a80e117ee66179c7f01f50d8c7d5e1e.jpg'], 'urls_thumb': ['https://img1.leboncoin.fr/ad-thumb/6165629f3db6dd424f4cdc4c24f4c748d3c74dd1.jpg', 'https://img0.leboncoin.fr/ad-thumb/982d50d10ae7b505e95c0b57a3b81b50b87d1ac8.jpg', 'https://img1.leboncoin.fr/ad-thumb/cbb97cd70a80e117ee66179c7f01f50d8c7d5e1e.jpg'], 'urls_large': ['https://img1.leboncoin.fr/ad-large/6165629f3db6dd424f4cdc4c24f4c748d3c74dd1.jpg', 'https://img0.leboncoin.fr/ad-large/982d50d10ae7b505e95c0b57a3b81b50b87d1ac8.jpg', 'https://img1.leboncoin.fr/ad-large/cbb97cd70a80e117ee66179c7f01f50d8c7d5e1e.jpg']}, 'url': 'https://www.leboncoin.fr/colocations/1700395664.htm', 'subject': 'Colocation une chambre Uniquement pour femme', 'index_date': '2019-11-02 23:46:11', 'expiration_date': '2020-01-01 23:46:11', 'first_publication_date': '2019-11-02 23:46:11'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '91', 'department_name': 'Essonne', 'city_label': 'Corbeil-Essonnes 91100', 'city': 'Corbeil-Essonnes', 'zipcode': '91100', 'lat': 48.61264, 'lng': 2.48232, 'source': 'city', 'provider': 'here', 'is_shape': True}, 'body': 'Bonjour \n\nJe met en location un bien de 25 m2 meublé à 5 min de la Gare rer Essonne Robinson. A proximité des commerces. \n\nNous acceptons les garantie type Visale, youse, garantme. \n\nDispo immédiatement.', 'images': {'thumb_url': 'https://img0.leboncoin.fr/ad-thumb/50578ede3d0a6f86366ae858ecef5e3c3f5c9222.jpg', 'small_url': 'https://img0.leboncoin.fr/ad-small/50578ede3d0a6f86366ae858ecef5e3c3f5c9222.jpg', 'nb_images': 3, 'urls': ['https://img0.leboncoin.fr/ad-image/50578ede3d0a6f86366ae858ecef5e3c3f5c9222.jpg', 'https://img7.leboncoin.fr/ad-image/07d5199f7e80427553e547cb7da5e601656e3c0c.jpg', 'https://img4.leboncoin.fr/ad-image/3455ff4be0f43260c2209dba6dee2c06525312f8.jpg'], 'urls_thumb': ['https://img0.leboncoin.fr/ad-thumb/50578ede3d0a6f86366ae858ecef5e3c3f5c9222.jpg', 'https://img7.leboncoin.fr/ad-thumb/07d5199f7e80427553e547cb7da5e601656e3c0c.jpg', 'https://img4.leboncoin.fr/ad-thumb/3455ff4be0f43260c2209dba6dee2c06525312f8.jpg'], 'urls_large': ['https://img0.leboncoin.fr/ad-large/50578ede3d0a6f86366ae858ecef5e3c3f5c9222.jpg', 'https://img7.leboncoin.fr/ad-large/07d5199f7e80427553e547cb7da5e601656e3c0c.jpg', 'https://img4.leboncoin.fr/ad-large/3455ff4be0f43260c2209dba6dee2c06525312f8.jpg']}, 'url': 'https://www.leboncoin.fr/locations/1700395600.htm', 'subject': '25 m2 récent meublé à 5 min de la gare RER', 'index_date': '2019-11-02 23:45:59', 'expiration_date': '2020-01-01 23:45:59', 'first_publication_date': '2019-11-02 23:45:59'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '94', 'department_name': 'Val-de-Marne', 'city_label': 'Arcueil 94110', 'city': 'Arcueil', 'zipcode': '94110', 'lat': 48.8119, 'lng': 2.33449, 'source': 'address', 'provider': 'here', 'is_shape': False}, 'body': "ARCUEIL. Appartement F4 de 78 m², refait à neuf, entièrement meublé et équipé, situé au 4ème étage d'un bel immeuble de standing construit en 2013, et bien entretenu. Très lumineux, il comprend une entrée desservant 3 chambres entre 9 et 12 m², une loggia, un petit balcon, une cuisine équipée haut de gamme ouverte sur un grand séjour salle à manger de 25 m², une salle de bain et une salle de douche avec WC. L'appartement bénéficie également de 2 places de parking en sous-sol (possibilité de louer la place 30€ par mois). Transports en communs accessibles à moins de 6 minutes à pied (RER B Laplace Maison des examens). Parc Montsouris à 15 minutes à pied. Quartier composé de nombreux commerces de proximité (dont marché, boulangerie et boucherie bio, pharmacie, supérette, maison de la presse, primeur, etc.) Nombreux rangements et grands dressing. Idéalement situé proche des écoles : Centre hospitalier vétérinaire Frégis, l'ENS, l'IUT, l'ESITC, l'ESTP et l'OSTEOBIO. Loyer mensuel de 620 euros la chambre charges comprises. DPE A.", 'images': {'thumb_url': 'https://img1.leboncoin.fr/ad-thumb/4ead9292094da9d207700df60bdea32d613d314c.jpg', 'small_url': 'https://img1.leboncoin.fr/ad-small/4ead9292094da9d207700df60bdea32d613d314c.jpg', 'nb_images': 3, 'urls': ['https://img1.leboncoin.fr/ad-image/4ead9292094da9d207700df60bdea32d613d314c.jpg', 'https://img7.leboncoin.fr/ad-image/1111f7d5c8a2eb24b811aa8187fafc7385a9fcea.jpg', 'https://img0.leboncoin.fr/ad-image/10c1e72e474ff5dace5ee4fde937db055cb8c79c.jpg'], 'urls_thumb': ['https://img1.leboncoin.fr/ad-thumb/4ead9292094da9d207700df60bdea32d613d314c.jpg', 'https://img7.leboncoin.fr/ad-thumb/1111f7d5c8a2eb24b811aa8187fafc7385a9fcea.jpg', 'https://img0.leboncoin.fr/ad-thumb/10c1e72e474ff5dace5ee4fde937db055cb8c79c.jpg'], 'urls_large': ['https://img1.leboncoin.fr/ad-large/4ead9292094da9d207700df60bdea32d613d314c.jpg', 'https://img7.leboncoin.fr/ad-large/1111f7d5c8a2eb24b811aa8187fafc7385a9fcea.jpg', 'https://img0.leboncoin.fr/ad-large/10c1e72e474ff5dace5ee4fde937db055cb8c79c.jpg']}, 'url': 'https://www.leboncoin.fr/colocations/1700395255.htm', 'subject': 'ARCUEIL - F4 Colocation 620cc', 'index_date': '2019-11-02 23:43:40', 'expiration_date': '2020-01-01 23:43:40', 'first_publication_date': '2019-11-02 23:43:40'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '93', 'department_name': 'Seine-Saint-Denis', 'city_label': 'Noisy-le-Grand 93160', 'city': 'Noisy-le-Grand', 'zipcode': '93160', 'lat': 48.83647, 'lng': 2.57063, 'source': 'address', 'provider': 'here', 'is_shape': False}, 'body': "Local situé 37/39 Allée du Closeau, Zone des Richardets, 93160 Noisy le Grand\nFaçade autoroute A4 . Accès RER Noisy le Grand à 10mn à pieds .\n\nDESCRIPTIF : Premier étage MONTE CHARGE 1 TONNE \nUn lot de 322 m2 de bureaux/ petite activité/ Show-room ou stockage, \nGrande salle carrée sans poteau ni cloison, au premier niveau \nLocal très lumineux: grande façade vitrée + sky-domes \nSanitaires H/F, Monte charge 1,2 tonne (1palette et transpalette),\nparkings privatifs.\n\nPRIX :\nLe loyer mensuel est de 1700€ ht x mois hors charges \nAu vu du détail des charges de l'exercice N-1, la provision pour charges mensuelle est de 850 € ht et comprend : \n- 3 parkings + l'entretien des espaces verts\n-La Taxe Foncière + la Taxe sur les Bureaux IDF\n-L'eau chaude et froide\n-Le chauffage\n-l’électricité\n-L'accès au restaurant d'entreprise .\n\nLocal disponible 30 SEPTEMBRE 2019\nBail 3/6/9 payable par trimestre d’avance + DG 3 mois HT", 'images': {'thumb_url': 'https://img6.leboncoin.fr/ad-thumb/63b1023bb9074adeeb41a8e26812bded1827df5a.jpg', 'small_url': 'https://img6.leboncoin.fr/ad-small/63b1023bb9074adeeb41a8e26812bded1827df5a.jpg', 'nb_images': 3, 'urls': ['https://img6.leboncoin.fr/ad-image/63b1023bb9074adeeb41a8e26812bded1827df5a.jpg', 'https://img3.leboncoin.fr/ad-image/a38969792dd72d2fa531f2c69fa4c19fd91efbef.jpg', 'https://img1.leboncoin.fr/ad-image/7b0a47fdb409be4f786cfc4da7701512fda0d938.jpg'], 'urls_thumb': ['https://img6.leboncoin.fr/ad-thumb/63b1023bb9074adeeb41a8e26812bded1827df5a.jpg', 'https://img3.leboncoin.fr/ad-thumb/a38969792dd72d2fa531f2c69fa4c19fd91efbef.jpg', 'https://img1.leboncoin.fr/ad-thumb/7b0a47fdb409be4f786cfc4da7701512fda0d938.jpg'], 'urls_large': ['https://img6.leboncoin.fr/ad-large/63b1023bb9074adeeb41a8e26812bded1827df5a.jpg', 'https://img3.leboncoin.fr/ad-large/a38969792dd72d2fa531f2c69fa4c19fd91efbef.jpg', 'https://img1.leboncoin.fr/ad-large/7b0a47fdb409be4f786cfc4da7701512fda0d938.jpg']}, 'url': 'https://www.leboncoin.fr/bureaux_commerces/1641782337.htm', 'subject': 'Loue plateau/ bureaux/ entrepôt/ activités', 'index_date': '2019-11-02 23:43:30', 'expiration_date': '2020-01-01 23:43:30', 'first_publication_date': '2019-11-02 23:43:30'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '75', 'department_name': 'Paris', 'city_label': 'Paris 75017', 'city': 'Paris', 'zipcode': '75017', 'lat': 48.89239, 'lng': 2.32456, 'source': 'address', 'provider': 'here', 'is_shape': False}, 'body': "english version bellow\n\nJe sous loue ma chambre du 8 novembre au 31 décembre (flexible) dans le cadre d'un détachement professionnel.\n\nGrande chambre, meublée, calme (cours intérieure) avec beaucoup de rangements. Appartement spacieux, meublé et tout équipé ( Lave-linge, four, fer à repasser, internet). L'immeuble est calme et se situe à 50m du metro Guy Moquet (ligne 13 et Bus 31). Le Prix inclue toutes les charges (eau, électricité, assurance, internet fibre, femme de ménage 2 fois par mois). Enfin, ma colloc est la plus sympa et elle aime faire la cuisine.\n\nNous cherchons une jeune femme, plutôt pro qu'étudiante (pour une question de rythme). Doit être en mesure de régler les deux mois de loyer à l'entrée dans l'appartement.\n\nI am subletting my room from novembre 8th until decembre 31st (flexible) while on a busniess trip. Large room with furnitures (and bedings if required), quiet and a lot of storage space. Spacious appartement all equiped (washing machine, oven, iron, internet). Very quiet building in a great area located 50m away from Guy Moquet Station (Line 13). Price includes everyting (water, electricity, insurance, highspeed internet, cleaning lady twice a month). My roomate is the best and she loves to cook.\n\nWe are looking for a young woman. Idealy a professionnal, if student don't plan on house parties during weekdays. Should be able to pay both months when moving in.", 'images': {'thumb_url': 'https://img4.leboncoin.fr/ad-thumb/9428c3720059e422f109da0db32a8bf798d44a1d.jpg', 'small_url': 'https://img4.leboncoin.fr/ad-small/9428c3720059e422f109da0db32a8bf798d44a1d.jpg', 'nb_images': 3, 'urls': ['https://img4.leboncoin.fr/ad-image/9428c3720059e422f109da0db32a8bf798d44a1d.jpg', 'https://img4.leboncoin.fr/ad-image/f29286c97f210da2cb3ba5b78574829aca6225e4.jpg', 'https://img0.leboncoin.fr/ad-image/46bce21e1f14c5e4209456a75707ace09f714ff6.jpg'], 'urls_thumb': ['https://img4.leboncoin.fr/ad-thumb/9428c3720059e422f109da0db32a8bf798d44a1d.jpg', 'https://img4.leboncoin.fr/ad-thumb/f29286c97f210da2cb3ba5b78574829aca6225e4.jpg', 'https://img0.leboncoin.fr/ad-thumb/46bce21e1f14c5e4209456a75707ace09f714ff6.jpg'], 'urls_large': ['https://img4.leboncoin.fr/ad-large/9428c3720059e422f109da0db32a8bf798d44a1d.jpg', 'https://img4.leboncoin.fr/ad-large/f29286c97f210da2cb3ba5b78574829aca6225e4.jpg', 'https://img0.leboncoin.fr/ad-large/46bce21e1f14c5e4209456a75707ace09f714ff6.jpg']}, 'url': 'https://www.leboncoin.fr/colocations/1700395193.htm', 'subject': 'Cherche coloc Femme, Meublé, 8 Nov- 31 Déc,Guy Moquet (17ème), 800/mois tout compris', 'index_date': '2019-11-02 23:43:12', 'expiration_date': '2020-01-01 23:43:12', 'first_publication_date': '2019-11-02 23:43:12'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '93', 'department_name': 'Seine-Saint-Denis', 'city_label': 'Noisy-le-Grand 93160', 'city': 'Noisy-le-Grand', 'zipcode': '93160', 'lat': 48.84887, 'lng': 2.55405, 'source': 'city', 'provider': 'here', 'is_shape': True}, 'body': "bonjour,\n\nje coloue 1 chambre pour 1 personne dans mon grand F4\n\nRER Noisy Le Grand Mont D'Est\n\nVous pouvez visiter le quartier avant de me contacter\n1 place Pablo Picasso\n93160 NOISY LE GRAND\n\nLa visite de l'appartement s'effectue tous les jours après 20h \n\nEn commun :\nSalon\nWC\nCuisine équipée\n\ncharges 30 eur : internet, eau, électricité\n\nMerci à vous\nK.", 'images': {'thumb_url': 'https://img2.leboncoin.fr/ad-thumb/6a234790bc5f4e2e958e84fd164d575477d40e45.jpg', 'small_url': 'https://img2.leboncoin.fr/ad-small/6a234790bc5f4e2e958e84fd164d575477d40e45.jpg', 'nb_images': 1, 'urls': ['https://img2.leboncoin.fr/ad-image/6a234790bc5f4e2e958e84fd164d575477d40e45.jpg'], 'urls_thumb': ['https://img2.leboncoin.fr/ad-thumb/6a234790bc5f4e2e958e84fd164d575477d40e45.jpg'], 'urls_large': ['https://img2.leboncoin.fr/ad-large/6a234790bc5f4e2e958e84fd164d575477d40e45.jpg']}, 'url': 'https://www.leboncoin.fr/colocations/1700394761.htm', 'subject': 'Colocation 1 chambre appartement F4 noisy le grand', 'index_date': '2019-11-02 23:40:29', 'expiration_date': '2020-01-01 23:40:29', 'first_publication_date': '2019-11-02 23:40:29'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '91', 'department_name': 'Essonne', 'city_label': 'Leudeville 91630', 'city': 'Leudeville', 'zipcode': '91630', 'lat': 48.56569, 'lng': 2.32619, 'source': 'city', 'provider': 'here', 'is_shape': True}, 'body': "EXCLUSIVITÉ : Dans quartier calme, petite copropriété, appartement 2p de 44m2 refait à neuf, \nentrée sur séjour, cuisine aménagée et équipée, 1 chambre, salle d'eau.\nGrenier aménagé en bureau et palier, 1 cave, 1 parking.", 'images': {'thumb_url': 'https://img0.leboncoin.fr/ad-thumb/7621cadf6749aed8b89040a0cfbfc89a7a1b5808.jpg', 'small_url': 'https://img0.leboncoin.fr/ad-small/7621cadf6749aed8b89040a0cfbfc89a7a1b5808.jpg', 'nb_images': 4, 'urls': ['https://img0.leboncoin.fr/ad-image/7621cadf6749aed8b89040a0cfbfc89a7a1b5808.jpg', 'https://img2.leboncoin.fr/ad-image/01f90e9d65783222a00503797d6eb7c8a7c91017.jpg', 'https://img5.leboncoin.fr/ad-image/f48e159a0e13c2209efda8a228d1355a4eb8c2fd.jpg', 'https://img5.leboncoin.fr/ad-image/c62961c022a5ad6d1562dad02c4472b842629ef8.jpg'], 'urls_thumb': ['https://img0.leboncoin.fr/ad-thumb/7621cadf6749aed8b89040a0cfbfc89a7a1b5808.jpg', 'https://img2.leboncoin.fr/ad-thumb/01f90e9d65783222a00503797d6eb7c8a7c91017.jpg', 'https://img5.leboncoin.fr/ad-thumb/f48e159a0e13c2209efda8a228d1355a4eb8c2fd.jpg', 'https://img5.leboncoin.fr/ad-thumb/c62961c022a5ad6d1562dad02c4472b842629ef8.jpg'], 'urls_large': ['https://img0.leboncoin.fr/ad-large/7621cadf6749aed8b89040a0cfbfc89a7a1b5808.jpg', 'https://img2.leboncoin.fr/ad-large/01f90e9d65783222a00503797d6eb7c8a7c91017.jpg', 'https://img5.leboncoin.fr/ad-large/f48e159a0e13c2209efda8a228d1355a4eb8c2fd.jpg', 'https://img5.leboncoin.fr/ad-large/c62961c022a5ad6d1562dad02c4472b842629ef8.jpg']}, 'url': 'https://www.leboncoin.fr/ventes_immobilieres/1700142841.htm', 'subject': 'Appartement 2 pièces LEUDEVILLE', 'index_date': '2019-11-02 23:40:03', 'expiration_date': '2020-01-01 23:40:03', 'first_publication_date': '2019-11-02 23:40:03'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '75', 'department_name': 'Paris', 'city_label': 'Paris 75017', 'city': 'Paris', 'zipcode': '75017', 'lat': 48.88387, 'lng': 2.31863, 'source': 'address', 'provider': 'here', 'is_shape': False}, 'body': 'Studio meublé 28 m²\n1 pièce\nMeublé\n1 salle d’eau\n1 WC\n5e étage\nAscenseur\n1 salon\n1 cuisine américaine\n1 salle de douche\nWC séparés\nLe chauffage est individuel électrique.\nCet appartement est disponible', 'images': {'thumb_url': 'https://img2.leboncoin.fr/ad-thumb/5599b0477bc1a46d7c319d36c2636fdf4186e567.jpg', 'small_url': 'https://img2.leboncoin.fr/ad-small/5599b0477bc1a46d7c319d36c2636fdf4186e567.jpg', 'nb_images': 3, 'urls': ['https://img2.leboncoin.fr/ad-image/5599b0477bc1a46d7c319d36c2636fdf4186e567.jpg', 'https://img2.leboncoin.fr/ad-image/98977fc744e04ee181b75a7568083b93143d9ea6.jpg', 'https://img2.leboncoin.fr/ad-image/060aa7d28041931536071f727483713cf7339920.jpg'], 'urls_thumb': ['https://img2.leboncoin.fr/ad-thumb/5599b0477bc1a46d7c319d36c2636fdf4186e567.jpg', 'https://img2.leboncoin.fr/ad-thumb/98977fc744e04ee181b75a7568083b93143d9ea6.jpg', 'https://img2.leboncoin.fr/ad-thumb/060aa7d28041931536071f727483713cf7339920.jpg'], 'urls_large': ['https://img2.leboncoin.fr/ad-large/5599b0477bc1a46d7c319d36c2636fdf4186e567.jpg', 'https://img2.leboncoin.fr/ad-large/98977fc744e04ee181b75a7568083b93143d9ea6.jpg', 'https://img2.leboncoin.fr/ad-large/060aa7d28041931536071f727483713cf7339920.jpg']}, 'url': 'https://www.leboncoin.fr/locations/1700394602.htm', 'subject': 'Très agréable studio meublé 28 m² (quartier Ternes)', 'index_date': '2019-11-02 23:39:40', 'expiration_date': '2020-01-01 23:39:40', 'first_publication_date': '2019-11-02 23:39:40'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '78', 'department_name': 'Yvelines', 'city_label': 'Buc 78530', 'city': 'Buc', 'zipcode': '78530', 'lat': 48.76767, 'lng': 2.11724, 'source': 'address', 'provider': 'here', 'is_shape': False}, 'body': "A louer, très bel appartement meublé tout équipé situé sur le haut de Buc, face à l'Intermarché. Appartement disponible dès le 1er décembre 2019. Adapté à personnel d'entreprise. Loyer : 1400 euros Charges comprise (eau, entretien de copro). \n\nL'appartement est très confortable : \nDécoré avec soin pour assurer votre confort et bien-être, il vous plaira par l'univers Modern design que nous avons créé.\nPratique, l’appartement est entièrement équipé avec tous les équipements assurant votre confort :\n- Une Cuisine tout équipée : Four, micro-onde, lave-vaisselle, lave-vaisselle, cafetière classique et Nespresso, Presseur de jus, grille pain, bouilloire, vaisselles \n- un coin salon avec un canapé et une télévision avec écran plat\n- une salle de bains avec douche, armoires et linge de toilette de luxe.\n- une chambre tout confort décorée avec goût.\n-d'un couloir avec 2 grandes armoires avec tous les équipements d'entretien\nVous profiterez aussi d'une place de parking privative protégée en sous-sol. Des places de parking sont disponibles à l'extérieur.\n\nL'appartement est idéalement situé, face à Intermarché de Buc : \n\nSitué face à la zone industrielle de Buc, l'appartement se trouve au carrefour d’activités professionnelles (Technocentre Renault, General Electric). Il est situé à moins de 10 minutes d'HEC Et du plateau de Saclay en voiture et de sites touristiques (Versailles/Château de Versailles, Golf de Buc, Golf National, Vallée de Chevreuse et de la Bièvre).\n\nSitué à 5 Km de la gare de la gare de Versailles Chantiers, il vous assurera des liaisons directes vers Paris (RER C, Train pour Montparnasse en 12 min, Train Defi pour la Défense en 23min). Il se trouve à 4 km de Versailles. L'appartement est proche du parc naturel régional de la Haute Vallée de Chevreuse, et se trouve proche de la vallée de la Bièvre.\n\nSi vous êtes intéressé, dossier à remettre avec pièces obligatoire (Pièce identité, garant, fiche de paie...). \nAucun démarchage agence accepté.", 'images': {'thumb_url': 'https://img7.leboncoin.fr/ad-thumb/7efc7a36fa73c62dd8fb599792ef98aa4a7dd46c.jpg', 'small_url': 'https://img7.leboncoin.fr/ad-small/7efc7a36fa73c62dd8fb599792ef98aa4a7dd46c.jpg', 'nb_images': 3, 'urls': ['https://img7.leboncoin.fr/ad-image/7efc7a36fa73c62dd8fb599792ef98aa4a7dd46c.jpg', 'https://img1.leboncoin.fr/ad-image/1934e8613bd8026267c045397881bc3679b647af.jpg', 'https://img6.leboncoin.fr/ad-image/671b7677742a00bae14d650c6847151b3817bc8c.jpg'], 'urls_thumb': ['https://img7.leboncoin.fr/ad-thumb/7efc7a36fa73c62dd8fb599792ef98aa4a7dd46c.jpg', 'https://img1.leboncoin.fr/ad-thumb/1934e8613bd8026267c045397881bc3679b647af.jpg', 'https://img6.leboncoin.fr/ad-thumb/671b7677742a00bae14d650c6847151b3817bc8c.jpg'], 'urls_large': ['https://img7.leboncoin.fr/ad-large/7efc7a36fa73c62dd8fb599792ef98aa4a7dd46c.jpg', 'https://img1.leboncoin.fr/ad-large/1934e8613bd8026267c045397881bc3679b647af.jpg', 'https://img6.leboncoin.fr/ad-large/671b7677742a00bae14d650c6847151b3817bc8c.jpg']}, 'url': 'https://www.leboncoin.fr/locations/1643536453.htm', 'subject': 'Location bel Appartement meublé/ tout équipé, très confort, au calme et au vert - Proche Versailles', 'index_date': '2019-11-02 23:39:05', 'expiration_date': '2020-01-01 23:39:05', 'first_publication_date': '2019-11-02 23:39:05'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '92', 'department_name': 'Hauts-de-Seine', 'city_label': 'Courbevoie 92400', 'city': 'Courbevoie', 'zipcode': '92400', 'lat': 48.89618, 'lng': 2.25648, 'source': 'city', 'provider': 'here', 'is_shape': True}, 'body': 'Au 6eme étage sans ascenseur, rue blondel, vue imprenable sur toits de courbevoie et ciel, immeuble année 30, entrée placard, cuisine frigo lave linge, living canapé convertible, sde avec WC et petits aménagements, F1 meublé, cave. Libre fin novembre début décembre.', 'images': {'thumb_url': 'https://img6.leboncoin.fr/ad-thumb/a75a9e0250e11dd8e3e6b37a88a4f09f6fc83c61.jpg', 'small_url': 'https://img6.leboncoin.fr/ad-small/a75a9e0250e11dd8e3e6b37a88a4f09f6fc83c61.jpg', 'nb_images': 9, 'urls': ['https://img6.leboncoin.fr/ad-image/a75a9e0250e11dd8e3e6b37a88a4f09f6fc83c61.jpg', 'https://img4.leboncoin.fr/ad-image/8bfff238989acbb84b20192bc12ed20df5c0d362.jpg', 'https://img5.leboncoin.fr/ad-image/ec617b1b2dfd0b0d746cf213fc40a256ab4307f3.jpg', 'https://img5.leboncoin.fr/ad-image/16916e5b4b986c1be31d10151903086313a89b4f.jpg', 'https://img5.leboncoin.fr/ad-image/96af63754e84ab29da6683b797c7f72728003189.jpg', 'https://img2.leboncoin.fr/ad-image/900407c5e90a0c6f03c8f169a01ce00738ae28b6.jpg', 'https://img1.leboncoin.fr/ad-image/ee0ebd4f2c66dc0aa2494a71e2a5f5d8e6eca2f6.jpg', 'https://img0.leboncoin.fr/ad-image/89019c266ab8fae3d058716e4a2b0c8714245b0e.jpg', 'https://img2.leboncoin.fr/ad-image/b18c04ee902c499d0564e537780b362d85b92e30.jpg'], 'urls_thumb': ['https://img6.leboncoin.fr/ad-thumb/a75a9e0250e11dd8e3e6b37a88a4f09f6fc83c61.jpg', 'https://img4.leboncoin.fr/ad-thumb/8bfff238989acbb84b20192bc12ed20df5c0d362.jpg', 'https://img5.leboncoin.fr/ad-thumb/ec617b1b2dfd0b0d746cf213fc40a256ab4307f3.jpg', 'https://img5.leboncoin.fr/ad-thumb/16916e5b4b986c1be31d10151903086313a89b4f.jpg', 'https://img5.leboncoin.fr/ad-thumb/96af63754e84ab29da6683b797c7f72728003189.jpg', 'https://img2.leboncoin.fr/ad-thumb/900407c5e90a0c6f03c8f169a01ce00738ae28b6.jpg', 'https://img1.leboncoin.fr/ad-thumb/ee0ebd4f2c66dc0aa2494a71e2a5f5d8e6eca2f6.jpg', 'https://img0.leboncoin.fr/ad-thumb/89019c266ab8fae3d058716e4a2b0c8714245b0e.jpg', 'https://img2.leboncoin.fr/ad-thumb/b18c04ee902c499d0564e537780b362d85b92e30.jpg'], 'urls_large': ['https://img6.leboncoin.fr/ad-large/a75a9e0250e11dd8e3e6b37a88a4f09f6fc83c61.jpg', 'https://img4.leboncoin.fr/ad-large/8bfff238989acbb84b20192bc12ed20df5c0d362.jpg', 'https://img5.leboncoin.fr/ad-large/ec617b1b2dfd0b0d746cf213fc40a256ab4307f3.jpg', 'https://img5.leboncoin.fr/ad-large/16916e5b4b986c1be31d10151903086313a89b4f.jpg', 'https://img5.leboncoin.fr/ad-large/96af63754e84ab29da6683b797c7f72728003189.jpg', 'https://img2.leboncoin.fr/ad-large/900407c5e90a0c6f03c8f169a01ce00738ae28b6.jpg', 'https://img1.leboncoin.fr/ad-large/ee0ebd4f2c66dc0aa2494a71e2a5f5d8e6eca2f6.jpg', 'https://img0.leboncoin.fr/ad-large/89019c266ab8fae3d058716e4a2b0c8714245b0e.jpg', 'https://img2.leboncoin.fr/ad-large/b18c04ee902c499d0564e537780b362d85b92e30.jpg']}, 'url': 'https://www.leboncoin.fr/locations/1699813476.htm', 'subject': 'Studio F1 courbevoie centre gare et la defense', 'index_date': '2019-11-02 23:36:24', 'expiration_date': '2019-12-31 23:51:10', 'first_publication_date': '2019-11-01 23:51:10'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '94', 'department_name': 'Val-de-Marne', 'city_label': 'Villeneuve-Saint-Georges 94190', 'city': 'Villeneuve-Saint-Georges', 'zipcode': '94190', 'lat': 48.73057, 'lng': 2.4615, 'source': 'address', 'provider': 'here', 'is_shape': False}, 'body': "Modèle ALISTAIR n°6 avec 3 chambres, superficie 81m2\n\n300 000 € TVA en 5,5% (residence principale, conditions nous consulter)\n\nCette maison est au cœur du DOMAINE D'ADELE, la 1ère résidence de 26 maisons BEPOS Effinergie 2017, passivhaus & E+C- (E3C2) de France. Maison haut de gamme répondant aux futur normes de construction RE2020. \n\nElle produit plus d‘énergie qu’elle n’en consomme grâce à des murs de 40 cm, des fenêtres securisées avec triple vitrage, une ventilation double flux pour un air sain, un chauffage granulé bois et 6 panneaux solaire posé sur la toiture végétalisée. \n\nLes prestations intérieures sont de très haute qualité (carrelage 60x60, wc suspendu geberit, portes haut de gamme, câble internet grade 3, tablier de baignoire carrelé, meuble vasque, volet roulant alu électrique, etc.). \n\nConstruite par le spécialiste et champion du BEPOS et du E+C-, nous vous garantissons par notre mode constructif en ossature bois passif économe en énergie, une maison exceptionnelle et conforme à la futur RE2020 dans une ville en pleine transformation ouverte sur l'avenir.\n\nSitué dans le meilleur quartier de Villeneuve, à 450 mètres de la future station de téléphérique bois-matar du CABLE A TELEVAL qui reliera la station de Metro Créteil - Pointe du Lac en 17 minutes dès 2021\n\ncharges d'énergie 280€ par an (dont 120€ de prime d’autoconsommation solaire pendant 5 ans) \nsoit 24€/mois pour l’ensemble de dépenses d’energie (électricité, abonnement, chauffage et eau chaude inclus).\n\nLa faible consommation et le chauffage bois vous garantit une maîtrise de votre budget sur le très long terme.\n\nDPE = -17 kwhEP/m2/AN (negatif) GES = 1\n\ncharge d'ASL pour les espace commun, l'entretien de la chaufferie, les portails, et le jardinage = 60-70€/mois\n\nfrais de notaire = 0€\nEligible au prêt à taux zéro\ndisponible au 1T 2020\n\nrenseignement : Eliane 06 63 63 23 04 ou Maxime 06 43 78 25 43\ncontact@ecolocost.com", 'images': {'thumb_url': 'https://img7.leboncoin.fr/ad-thumb/40b207b8c1290556d3b2065bef37c553d55f40e7.jpg', 'small_url': 'https://img7.leboncoin.fr/ad-small/40b207b8c1290556d3b2065bef37c553d55f40e7.jpg', 'nb_images': 5, 'urls': ['https://img7.leboncoin.fr/ad-image/40b207b8c1290556d3b2065bef37c553d55f40e7.jpg', 'https://img5.leboncoin.fr/ad-image/3097eb633e8759041c72d9a68eedd67f1a962b28.jpg', 'https://img6.leboncoin.fr/ad-image/16939a0b45a38cc64a7a86ee7ba51d08f75af50c.jpg', 'https://img3.leboncoin.fr/ad-image/7a3e183ed6f0cc8b1d40e7a8c9898c8579354287.jpg', 'https://img7.leboncoin.fr/ad-image/d1d9470f7ef61cdfa2bf5652b2da8745d5262a4c.jpg'], 'urls_thumb': ['https://img7.leboncoin.fr/ad-thumb/40b207b8c1290556d3b2065bef37c553d55f40e7.jpg', 'https://img5.leboncoin.fr/ad-thumb/3097eb633e8759041c72d9a68eedd67f1a962b28.jpg', 'https://img6.leboncoin.fr/ad-thumb/16939a0b45a38cc64a7a86ee7ba51d08f75af50c.jpg', 'https://img3.leboncoin.fr/ad-thumb/7a3e183ed6f0cc8b1d40e7a8c9898c8579354287.jpg', 'https://img7.leboncoin.fr/ad-thumb/d1d9470f7ef61cdfa2bf5652b2da8745d5262a4c.jpg'], 'urls_large': ['https://img7.leboncoin.fr/ad-large/40b207b8c1290556d3b2065bef37c553d55f40e7.jpg', 'https://img5.leboncoin.fr/ad-large/3097eb633e8759041c72d9a68eedd67f1a962b28.jpg', 'https://img6.leboncoin.fr/ad-large/16939a0b45a38cc64a7a86ee7ba51d08f75af50c.jpg', 'https://img3.leboncoin.fr/ad-large/7a3e183ed6f0cc8b1d40e7a8c9898c8579354287.jpg', 'https://img7.leboncoin.fr/ad-large/d1d9470f7ef61cdfa2bf5652b2da8745d5262a4c.jpg']}, 'url': 'https://www.leboncoin.fr/ventes_immobilieres/1656516542.htm', 'subject': 'Maison positive en energie dans un domaine privé', 'index_date': '2019-11-02 23:32:36', 'expiration_date': '2019-12-07 00:26:34', 'first_publication_date': '2019-10-08 00:26:34'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '77', 'department_name': 'Seine-et-Marne', 'city_label': 'Roissy-en-Brie 77680', 'city': 'Roissy-en-Brie', 'zipcode': '77680', 'lat': 48.79024, 'lng': 2.65431, 'source': 'city', 'provider': 'here', 'is_shape': True}, 'body': "Vend Fond de commerce restaurant Sandwicherie,Pizzeria, Tacos, Kebab, Burger situe en face de la gare de Roissy en Brie ERE E, lycée, collège sont à 400 m, le restaurant très propre matériels aussi.\nBail 3/6/9, surface 120 m2, environ 32 places assises , adresse 2 place de la gare 77680 Roissy en Brie\nNous contacter si vous êtes intéressé.\nPour plus d'information n'hésitez pas de me contacter 06 12 81 99 26", 'images': {'thumb_url': 'https://img4.leboncoin.fr/ad-thumb/f3e7c75484d9a0d52f5857a63f13f141a6eb608b.jpg', 'small_url': 'https://img4.leboncoin.fr/ad-small/f3e7c75484d9a0d52f5857a63f13f141a6eb608b.jpg', 'nb_images': 3, 'urls': ['https://img4.leboncoin.fr/ad-image/f3e7c75484d9a0d52f5857a63f13f141a6eb608b.jpg', 'https://img5.leboncoin.fr/ad-image/43dba39447ada6a95383ddac8f342dca8865c86c.jpg', 'https://img2.leboncoin.fr/ad-image/82557b185bf7f2493c6eac7d56f79f7802847c1e.jpg'], 'urls_thumb': ['https://img4.leboncoin.fr/ad-thumb/f3e7c75484d9a0d52f5857a63f13f141a6eb608b.jpg', 'https://img5.leboncoin.fr/ad-thumb/43dba39447ada6a95383ddac8f342dca8865c86c.jpg', 'https://img2.leboncoin.fr/ad-thumb/82557b185bf7f2493c6eac7d56f79f7802847c1e.jpg'], 'urls_large': ['https://img4.leboncoin.fr/ad-large/f3e7c75484d9a0d52f5857a63f13f141a6eb608b.jpg', 'https://img5.leboncoin.fr/ad-large/43dba39447ada6a95383ddac8f342dca8865c86c.jpg', 'https://img2.leboncoin.fr/ad-large/82557b185bf7f2493c6eac7d56f79f7802847c1e.jpg']}, 'url': 'https://www.leboncoin.fr/bureaux_commerces/1675857837.htm', 'subject': 'Fond de commerce restauration rapide', 'index_date': '2019-11-02 23:32:19', 'expiration_date': '2019-11-14 22:48:20', 'first_publication_date': '2019-09-15 22:48:20'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '93', 'department_name': 'Seine-Saint-Denis', 'city_label': 'Livry-Gargan 93190', 'city': 'Livry-Gargan', 'zipcode': '93190', 'lat': 48.90576, 'lng': 2.51853, 'source': 'address', 'provider': 'here', 'is_shape': False}, 'body': "Box a louer Livry-gargan quartier Chanzy Gutemberg tramway a pied de l'immeuble\n100€ charges comprises + caution pour le bip", 'images': {'nb_images': 0}, 'url': 'https://www.leboncoin.fr/locations/1700393393.htm', 'subject': 'Box a louer sécurisé', 'index_date': '2019-11-02 23:32:16', 'expiration_date': '2020-01-01 23:32:16', 'first_publication_date': '2019-11-02 23:32:16'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '93', 'department_name': 'Seine-Saint-Denis', 'city_label': 'Le Raincy 93340', 'city': 'Le Raincy', 'zipcode': '93340', 'lat': 48.8977203369, 'lng': 2.5160799026, 'source': 'city', 'provider': 'here', 'is_shape': True}, 'body': 'Loue place de parking avec porte automatique située avenue de la Résistance à proximité immédiate de Carrefour City.\nDimensions 2,80 m x 5,00 m hauteur 1,95 m\nPremier contact uniquement par mail\nLibre au 01 décembre\nPrix : 90 € charges comprises', 'images': {'thumb_url': 'https://img6.leboncoin.fr/ad-thumb/3b2720a5e8e508407525b8c485c8d33341d8781c.jpg', 'small_url': 'https://img6.leboncoin.fr/ad-small/3b2720a5e8e508407525b8c485c8d33341d8781c.jpg', 'nb_images': 1, 'urls': ['https://img6.leboncoin.fr/ad-image/3b2720a5e8e508407525b8c485c8d33341d8781c.jpg'], 'urls_thumb': ['https://img6.leboncoin.fr/ad-thumb/3b2720a5e8e508407525b8c485c8d33341d8781c.jpg'], 'urls_large': ['https://img6.leboncoin.fr/ad-large/3b2720a5e8e508407525b8c485c8d33341d8781c.jpg']}, 'url': 'https://www.leboncoin.fr/locations/1700393091.htm', 'subject': 'Place de parking en sous-sol', 'index_date': '2019-11-02 23:30:43', 'expiration_date': '2020-01-01 23:30:43', 'first_publication_date': '2019-11-02 23:30:43'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '95', 'department_name': "Val-d'Oise", 'city_label': 'Soisy-sous-Montmorency 95230', 'city': 'Soisy-sous-Montmorency', 'zipcode': '95230', 'lat': 48.98754, 'lng': 2.30085, 'source': 'user', 'provider': 'lbc', 'is_shape': False}, 'body': "Chambre meublée en colocation à louer dans l'appartement à soisy sous Montmorency 95\nProche toutes commodités et gare à 5 mns à pied. \nGare du Nord ligne h à 15 mns au prix de 350 euros cc.\n\nSi vous êtes intéressé merci de me contacter au 06.52.64.79.24\n\nPas sérieux s'abstenir", 'images': {'nb_images': 0}, 'url': 'https://www.leboncoin.fr/colocations/1700392966.htm', 'subject': 'Chambre meublée', 'index_date': '2019-11-02 23:29:58', 'expiration_date': '2020-01-01 23:29:58', 'first_publication_date': '2019-11-02 23:29:58'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '94', 'department_name': 'Val-de-Marne', 'city_label': 'Gentilly 94250', 'city': 'Gentilly', 'zipcode': '94250', 'lat': 48.81518, 'lng': 2.34861, 'source': 'city', 'provider': 'here', 'is_shape': True}, 'body': "Bonjour\n\nA votre disposition un Fond de Commerce de Pressing, Blanchisserie -Teinturerie, Mercerie, Relais Colis la Redoute. Bon emplacement. Très belle affaire avec bonne clientèle et une bonne réputation avec un travail de qualité.\n\nOuvert de 8h à 19h30. Idéal pour un couple. C'est une très belle opportunité avec un potentiel de développement. Très bel agencement, grande fonctionnalité. Superficie d'environ 120m2 + une cave de 25 m2. \n\nSi vous êtes intéressé par ce local, nous pouvons aussi le laisser comme fond de commerce vide. Libre à vous d'en faire une agence de voyages, une jeune entreprise , Bazar, Fleuriste ? Agence de voyage , Boutique de décoration, Pâtisserie orientale, poissonneries, Bureau d’étude Parfumerie, Crêperie, opticiens ou ce qui vous plaira. Nous nous occuperons de vous laisser le local à vide.\n\nBel emplacement pour tout Commerces sauf restaurations, Épicerie, salon de coiffure car sur le site il y en a déjà Le loyer est de 1100euros par mois avec charges comprises ! Matériel et agencement en bon état.\n\nVous pouvez créer votre société Startup,\nPièces de rechanges autos motos\nTaxi phones\nReparation ordinateurs + téléphones portables\nAgence de voyage et bien d'autres activités \n\nPour tout renseignement complémentaire n'hésitez pas à nous contacter. \n\nVenez visiter, c'est une très belle affaire à ne pas rater.\n\nAvec un très grand regret que je cede mon fond de commerce cause SANTÉ .....!!!!!\n\nN,hésitez le prix est négociable dans la mesure du raisonnable \n\nAlors n,attendez pas trop c,est une très bonne affaire", 'images': {'thumb_url': 'https://img5.leboncoin.fr/ad-thumb/dcf9479682bb53e2c11a67c808a46a6a723019d7.jpg', 'small_url': 'https://img5.leboncoin.fr/ad-small/dcf9479682bb53e2c11a67c808a46a6a723019d7.jpg', 'nb_images': 3, 'urls': ['https://img5.leboncoin.fr/ad-image/dcf9479682bb53e2c11a67c808a46a6a723019d7.jpg', 'https://img7.leboncoin.fr/ad-image/19df304d43e162f79cda0b2d33a29d1d0ad06cb6.jpg', 'https://img7.leboncoin.fr/ad-image/cde8d15b99d0d85323bafac3031da3464a44af42.jpg'], 'urls_thumb': ['https://img5.leboncoin.fr/ad-thumb/dcf9479682bb53e2c11a67c808a46a6a723019d7.jpg', 'https://img7.leboncoin.fr/ad-thumb/19df304d43e162f79cda0b2d33a29d1d0ad06cb6.jpg', 'https://img7.leboncoin.fr/ad-thumb/cde8d15b99d0d85323bafac3031da3464a44af42.jpg'], 'urls_large': ['https://img5.leboncoin.fr/ad-large/dcf9479682bb53e2c11a67c808a46a6a723019d7.jpg', 'https://img7.leboncoin.fr/ad-large/19df304d43e162f79cda0b2d33a29d1d0ad06cb6.jpg', 'https://img7.leboncoin.fr/ad-large/cde8d15b99d0d85323bafac3031da3464a44af42.jpg']}, 'url': 'https://www.leboncoin.fr/bureaux_commerces/1073173880.htm', 'subject': 'Fond Commerce 130 m2 proche Paris RER B', 'index_date': '2019-11-02 23:29:20', 'expiration_date': '2020-01-01 23:29:20', 'first_publication_date': '2019-11-02 23:29:20'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '75', 'department_name': 'Paris', 'city_label': 'Paris 75018', 'city': 'Paris', 'zipcode': '75018', 'lat': 48.892490387, 'lng': 2.3446600437, 'source': 'address', 'provider': 'here', 'is_shape': False}, 'body': 'Sous-location à louer du 21 décembre 2019 au 5 janvier 2020\n\nChambre confortable dans un appartement Haussmann, 6ème étage avec beaucoup de lumière. Appartement composé de deux chambres (colocation), salon, salle de bain et cuisine. Chauffage partout.\n\nQuartier animé avec plusieurs supermarchés accessibles à pied, bars, restaurants, marchés, Sacré Coeur et Montmartre, Les Marches aux Puces, etc. Accessible par les lignes de métro 4 (5 minutes à pied) ou 12 (7 minutes à pied) et de nombreuses lignes de bus.\n\nLe loyer est de 350 Euro(s) TTC, + caution. Les visites possibles, en fait, sont préférées. Premier arrivé premier servi. Si vous souhaitez visiter plus tôt, vous pouvez effectuer un dépôt et garantir la chambre à vous!', 'images': {'nb_images': 0}, 'url': 'https://www.leboncoin.fr/colocations/1700391916.htm', 'subject': 'Sous-location vers 21 décembre au 5 janvier', 'index_date': '2019-11-02 23:24:56', 'expiration_date': '2020-01-01 23:24:56', 'first_publication_date': '2019-11-02 23:24:56'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '77', 'department_name': 'Seine-et-Marne', 'city_label': 'Meaux 77100', 'city': 'Meaux', 'zipcode': '77100', 'lat': 48.95936, 'lng': 2.88306, 'source': 'city', 'provider': 'here', 'is_shape': True}, 'body': "Bonjour je recherche une maison t4, je suis actuellement au rsa avec 2 enfants à charge et enceinte de mon troisième enfants.\nJe suis à jour de mes loyers ! \nMerci de ne pas juger ma situation...\nChers propriétaires vous avez la garantie d'avoir une personne qui paie ses loyers tous les mois attestations de mes précédents propriétaires !! Une personne en CDI n'est pas obligatoirement les meilleurs payeurs .\nApl est directement versée (une garantie d'avoir une partie du loyer)et bien entendu je verserai le reste !\nJe recherche au sud du 02, au Nord du 77 95 et au sud du 60\n\nA réfléchir ! \n\nCordialement", 'images': {'nb_images': 0}, 'url': 'https://www.leboncoin.fr/locations/1700391431.htm', 'subject': 'Maison t4', 'index_date': '2019-11-02 23:22:37', 'expiration_date': '2020-01-01 23:22:37', 'first_publication_date': '2019-11-02 23:22:37'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '77', 'department_name': 'Seine-et-Marne', 'city_label': 'Moissy-Cramayel 77550', 'city': 'Moissy-Cramayel', 'zipcode': '77550', 'lat': 48.6326789856, 'lng': 2.5847001076, 'source': 'address', 'provider': 'here', 'is_shape': False}, 'body': 'Place de parking en souterrain dans une résidence fermé et sécurisé, avenue de chanteloup à moissy cramayel.\nAvec un boîtier qui ouvre le portail de la résidence et la porte du parking.', 'images': {'thumb_url': 'https://img4.leboncoin.fr/ad-thumb/d75714f8d916487d5e1f03b5ec53a3c9dc058cd7.jpg', 'small_url': 'https://img4.leboncoin.fr/ad-small/d75714f8d916487d5e1f03b5ec53a3c9dc058cd7.jpg', 'nb_images': 1, 'urls': ['https://img4.leboncoin.fr/ad-image/d75714f8d916487d5e1f03b5ec53a3c9dc058cd7.jpg'], 'urls_thumb': ['https://img4.leboncoin.fr/ad-thumb/d75714f8d916487d5e1f03b5ec53a3c9dc058cd7.jpg'], 'urls_large': ['https://img4.leboncoin.fr/ad-large/d75714f8d916487d5e1f03b5ec53a3c9dc058cd7.jpg']}, 'url': 'https://www.leboncoin.fr/locations/1700391014.htm', 'subject': 'Place de parking souterrain', 'index_date': '2019-11-02 23:20:46', 'expiration_date': '2020-01-01 23:20:46', 'first_publication_date': '2019-11-02 23:20:46'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '75', 'department_name': 'Paris', 'city_label': 'Paris 75020', 'city': 'Paris', 'zipcode': '75020', 'lat': 48.87228, 'lng': 2.38765, 'source': 'address', 'provider': 'here', 'is_shape': False}, 'body': "Situé entre Pyrénées et Jourdain, à 2 minutes à pied du Parc de Belleville et de nombreux commerces, appartement T3 traversant de 60m² au sein d'une copropriété familiale et bien entretenue (ascenseur, gardien).\n\n2ème étage, très calme, lumineux et sans vis-à-vis.\n\nL'appartement est composé d'une entrée, d'une grande pièce à vivre (25m² environ) avec cuisine ouverte, de 2 chambres spacieuses, d'une salle de bain et de wc séparés. Nombreux placards et aucune perte de place.\n\nDiagnostics en cours.", 'images': {'thumb_url': 'https://img1.leboncoin.fr/ad-thumb/77f37b21b8d95f1d6f7f5cbc41d6c73febb52153.jpg', 'small_url': 'https://img1.leboncoin.fr/ad-small/77f37b21b8d95f1d6f7f5cbc41d6c73febb52153.jpg', 'nb_images': 5, 'urls': ['https://img1.leboncoin.fr/ad-image/77f37b21b8d95f1d6f7f5cbc41d6c73febb52153.jpg', 'https://img4.leboncoin.fr/ad-image/5961b771a61017111dd286c818978859dc753d11.jpg', 'https://img7.leboncoin.fr/ad-image/259dbbf94e289fb8802fe1a00eadaf0a7c351a85.jpg', 'https://img2.leboncoin.fr/ad-image/d15ab0f472b99fcb5cd34d700c84f23001d6f82f.jpg', 'https://img4.leboncoin.fr/ad-image/e966560865082c76cd77627421d4e32e86233c9a.jpg'], 'urls_thumb': ['https://img1.leboncoin.fr/ad-thumb/77f37b21b8d95f1d6f7f5cbc41d6c73febb52153.jpg', 'https://img4.leboncoin.fr/ad-thumb/5961b771a61017111dd286c818978859dc753d11.jpg', 'https://img7.leboncoin.fr/ad-thumb/259dbbf94e289fb8802fe1a00eadaf0a7c351a85.jpg', 'https://img2.leboncoin.fr/ad-thumb/d15ab0f472b99fcb5cd34d700c84f23001d6f82f.jpg', 'https://img4.leboncoin.fr/ad-thumb/e966560865082c76cd77627421d4e32e86233c9a.jpg'], 'urls_large': ['https://img1.leboncoin.fr/ad-large/77f37b21b8d95f1d6f7f5cbc41d6c73febb52153.jpg', 'https://img4.leboncoin.fr/ad-large/5961b771a61017111dd286c818978859dc753d11.jpg', 'https://img7.leboncoin.fr/ad-large/259dbbf94e289fb8802fe1a00eadaf0a7c351a85.jpg', 'https://img2.leboncoin.fr/ad-large/d15ab0f472b99fcb5cd34d700c84f23001d6f82f.jpg', 'https://img4.leboncoin.fr/ad-large/e966560865082c76cd77627421d4e32e86233c9a.jpg']}, 'url': 'https://www.leboncoin.fr/ventes_immobilieres/1688283297.htm', 'subject': 'Appartement 60m2 Paris 20', 'index_date': '2019-11-02 23:20:17', 'expiration_date': '2019-12-09 18:41:54', 'first_publication_date': '2019-10-10 18:41:54'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '75', 'department_name': 'Paris', 'city_label': 'Paris 75018', 'city': 'Paris', 'zipcode': '75018', 'lat': 48.89018, 'lng': 2.36091, 'source': 'address', 'provider': 'here', 'is_shape': False}, 'body': "App 2 pièces clair, calme, refait à neuf en 11/2017.\nChambre lit 2pl, armoire, commode, bureau.\nSalon/cuisine canapé convertible, commode, plaques, four, hotte, placards.\nSalle d'eau et WC.\nMetro, bus, Velib, taxis et tous commerces à proximité.\nConvient couple, colocation possible.\n\nPossibilité autres prestations : machine à laver séchante, vaisselle et litterie.", 'images': {'thumb_url': 'https://img7.leboncoin.fr/ad-thumb/7034624513a8e03081fa62de4f590219543d2528.jpg', 'small_url': 'https://img7.leboncoin.fr/ad-small/7034624513a8e03081fa62de4f590219543d2528.jpg', 'nb_images': 10, 'urls': ['https://img7.leboncoin.fr/ad-image/7034624513a8e03081fa62de4f590219543d2528.jpg', 'https://img7.leboncoin.fr/ad-image/fb07309b4883a9bb8b230a93f7c680e541ec5315.jpg', 'https://img5.leboncoin.fr/ad-image/a14ae23ac86bfb41c4580f7e2e5b76f7dadf5785.jpg', 'https://img6.leboncoin.fr/ad-image/d264fb7630f6a8f9098380568b62d1802f10b203.jpg', 'https://img1.leboncoin.fr/ad-image/40a65d72763d13ae990c3255f1c7664f8b742a27.jpg', 'https://img7.leboncoin.fr/ad-image/a61b7c810c38b82bc82dd2e4a0bda7c905ad45c4.jpg', 'https://img1.leboncoin.fr/ad-image/6888879fbfd0d76cd60a343c234cc19a691bfbde.jpg', 'https://img3.leboncoin.fr/ad-image/deb24047da3f1af387034a38f941f554858f8d5e.jpg', 'https://img6.leboncoin.fr/ad-image/911a92f11ad0533976e61376824a4eb270d34cd1.jpg', 'https://img3.leboncoin.fr/ad-image/e744ad34d7317b07321b68130672dc595a6d6c8f.jpg'], 'urls_thumb': ['https://img7.leboncoin.fr/ad-thumb/7034624513a8e03081fa62de4f590219543d2528.jpg', 'https://img7.leboncoin.fr/ad-thumb/fb07309b4883a9bb8b230a93f7c680e541ec5315.jpg', 'https://img5.leboncoin.fr/ad-thumb/a14ae23ac86bfb41c4580f7e2e5b76f7dadf5785.jpg', 'https://img6.leboncoin.fr/ad-thumb/d264fb7630f6a8f9098380568b62d1802f10b203.jpg', 'https://img1.leboncoin.fr/ad-thumb/40a65d72763d13ae990c3255f1c7664f8b742a27.jpg', 'https://img7.leboncoin.fr/ad-thumb/a61b7c810c38b82bc82dd2e4a0bda7c905ad45c4.jpg', 'https://img1.leboncoin.fr/ad-thumb/6888879fbfd0d76cd60a343c234cc19a691bfbde.jpg', 'https://img3.leboncoin.fr/ad-thumb/deb24047da3f1af387034a38f941f554858f8d5e.jpg', 'https://img6.leboncoin.fr/ad-thumb/911a92f11ad0533976e61376824a4eb270d34cd1.jpg', 'https://img3.leboncoin.fr/ad-thumb/e744ad34d7317b07321b68130672dc595a6d6c8f.jpg'], 'urls_large': ['https://img7.leboncoin.fr/ad-large/7034624513a8e03081fa62de4f590219543d2528.jpg', 'https://img7.leboncoin.fr/ad-large/fb07309b4883a9bb8b230a93f7c680e541ec5315.jpg', 'https://img5.leboncoin.fr/ad-large/a14ae23ac86bfb41c4580f7e2e5b76f7dadf5785.jpg', 'https://img6.leboncoin.fr/ad-large/d264fb7630f6a8f9098380568b62d1802f10b203.jpg', 'https://img1.leboncoin.fr/ad-large/40a65d72763d13ae990c3255f1c7664f8b742a27.jpg', 'https://img7.leboncoin.fr/ad-large/a61b7c810c38b82bc82dd2e4a0bda7c905ad45c4.jpg', 'https://img1.leboncoin.fr/ad-large/6888879fbfd0d76cd60a343c234cc19a691bfbde.jpg', 'https://img3.leboncoin.fr/ad-large/deb24047da3f1af387034a38f941f554858f8d5e.jpg', 'https://img6.leboncoin.fr/ad-large/911a92f11ad0533976e61376824a4eb270d34cd1.jpg', 'https://img3.leboncoin.fr/ad-large/e744ad34d7317b07321b68130672dc595a6d6c8f.jpg']}, 'url': 'https://www.leboncoin.fr/locations/1700386307.htm', 'subject': 'App T2 meublé', 'index_date': '2019-11-02 23:14:43', 'expiration_date': '2020-01-01 23:14:43', 'first_publication_date': '2019-11-02 23:14:43'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '77', 'department_name': 'Seine-et-Marne', 'city_label': 'Nemours 77140', 'city': 'Nemours', 'zipcode': '77140', 'lat': 48.26722, 'lng': 2.69473, 'source': 'city', 'provider': 'here', 'is_shape': True}, 'body': "Bonjour,\n\nExpert en terrassement et aménagement extérieur propose ses services pour :\n\n- Tous type de Terrassement, Terrassier\n- Démolition\n-Préparation pour construction\n-Tranchées\n- Installation de fosse septique,\n- Terrassement de masse ( nivellement de terrain )\n- dessouchage\n- Pose de Piscine coque\n- pose de terrasse bois\n- Création d'allées, chemin d'accès\n- Pose de Gazon Synthétique\n- Construction Abri de voiture, abri de jardin\n- mini-pelle, \n\nDEVIS rapide et gratuit sous 48h\n\nTEL 07 82 65 57 13", 'images': {'thumb_url': 'https://img7.leboncoin.fr/ad-thumb/8e61a0c309abadd79fa1c83aec633ab1aa414b5e.jpg', 'small_url': 'https://img7.leboncoin.fr/ad-small/8e61a0c309abadd79fa1c83aec633ab1aa414b5e.jpg', 'nb_images': 3, 'urls': ['https://img7.leboncoin.fr/ad-image/8e61a0c309abadd79fa1c83aec633ab1aa414b5e.jpg', 'https://img5.leboncoin.fr/ad-image/4ec4e7764f8c65c534d03f7bb762f1175b3c1678.jpg', 'https://img1.leboncoin.fr/ad-image/37331d4f0af2321bc2c9eabcaab83d34126daf01.jpg'], 'urls_thumb': ['https://img7.leboncoin.fr/ad-thumb/8e61a0c309abadd79fa1c83aec633ab1aa414b5e.jpg', 'https://img5.leboncoin.fr/ad-thumb/4ec4e7764f8c65c534d03f7bb762f1175b3c1678.jpg', 'https://img1.leboncoin.fr/ad-thumb/37331d4f0af2321bc2c9eabcaab83d34126daf01.jpg'], 'urls_large': ['https://img7.leboncoin.fr/ad-large/8e61a0c309abadd79fa1c83aec633ab1aa414b5e.jpg', 'https://img5.leboncoin.fr/ad-large/4ec4e7764f8c65c534d03f7bb762f1175b3c1678.jpg', 'https://img1.leboncoin.fr/ad-large/37331d4f0af2321bc2c9eabcaab83d34126daf01.jpg']}, 'url': 'https://www.leboncoin.fr/ventes_immobilieres/1700389067.htm', 'subject': 'Terrassier', 'index_date': '2019-11-02 23:12:20', 'expiration_date': '2020-01-01 23:12:20', 'first_publication_date': '2019-11-02 23:12:20'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '91', 'department_name': 'Essonne', 'city_label': 'Massy 91300', 'city': 'Massy', 'zipcode': '91300', 'lat': 48.73364, 'lng': 2.28572, 'source': 'address', 'provider': 'here', 'is_shape': False}, 'body': "Je vous propose un bien ideal pour colocation dans un appartement spacieux entièrement refait à neuf, situé au 1er étage dans une residence bien entretenu.\n\n========================================================\n🏡 Description:\n========================================================\n- Cuisine Americaine entièrement équipée: four électrique 🥘, plaques de cuisson induction 4 feux 🍳, micro-onde 🍿, grand réfrigérateur-congélateur avec distributeur eau fraiche ❄️\U0001f976 etc.\n- Salon avec TV, grand canapé, table et chaises, rideaux double etc.\n- Chaque chambre est équipée d'un grand 🛏️ double, parure de lit, bureaux 🤓, lampe de bureau, étagères 📚, dressing, rideaux double.\n- Salle de bain entièrement rénovée , avec 🛀 et miroir anti buée \n- Grande cave (9m2)\n- Accès internet + cable + téléphone (inclus dans le loyer)\n\n========================================================\nMoyens de transport:\n========================================================\n- 🚅 Gare RER B Les Baconnets - 10 min à pied (ou 40 min de Paris et 40 min de l'Université Paris-sud)\n- 🚍 BUS: 197, 119, 319 et N21 - (3-5 min à pied)\n- 🚶5 min à pied Carrefour, Franprix, pharmacie, boulangerie, cinema etc. \n- 🚗 accès rapide à l'autoroutes A10, A6 et N20.\n- ✈️ 30 minute de l'aéroport d'Orly\n\n========================================================\nDétails pratiques 🤑:\n========================================================\nUne chambre sur 3 est déjà occupée. Il reste encore 2 chambres disponibles:\nChambre 1: 600 € (toutes charges comprises)\nChambre 2: 650 € (toutes charges comprises)\nCharges inclus: électricité, internet, eau chaude et froide, chauffage au sol collectif 🏆 \nDetails contrat: contrat individuel par colocataire\nDisponibilité: 1er Novembre ⏳\n\n========================================================\nIntéressé(e)? 👀\n========================================================\nPour plus d’informations, merci de me contacter", 'images': {'thumb_url': 'https://img5.leboncoin.fr/ad-thumb/72dde7ad3585ed4ad2e98318bbb8db013a8c663a.jpg', 'small_url': 'https://img5.leboncoin.fr/ad-small/72dde7ad3585ed4ad2e98318bbb8db013a8c663a.jpg', 'nb_images': 9, 'urls': ['https://img5.leboncoin.fr/ad-image/72dde7ad3585ed4ad2e98318bbb8db013a8c663a.jpg', 'https://img4.leboncoin.fr/ad-image/05fce9cdd7bd015d53b2c95e4cb3e3ca83010542.jpg', 'https://img5.leboncoin.fr/ad-image/1382fa897e2c25ed323fb4dfed87504b9fdacfae.jpg', 'https://img1.leboncoin.fr/ad-image/0366debe46a69eaf9ac9eee15f17150e5a49ec54.jpg', 'https://img2.leboncoin.fr/ad-image/7828308b93335be95aaf010598c736a74bb8851b.jpg', 'https://img1.leboncoin.fr/ad-image/1cb8a1b92b65f9fe5cc2eefd4d7d0daf8ec1cd98.jpg', 'https://img6.leboncoin.fr/ad-image/ae2b9eba587ce402ef374ac3b9868ff54bc5dd6c.jpg', 'https://img2.leboncoin.fr/ad-image/7c38a4cccd32f0a5537d79132e23acfc7eda2565.jpg', 'https://img3.leboncoin.fr/ad-image/67d555313224a8e88575eabadbe67e2f63ee35bc.jpg'], 'urls_thumb': ['https://img5.leboncoin.fr/ad-thumb/72dde7ad3585ed4ad2e98318bbb8db013a8c663a.jpg', 'https://img4.leboncoin.fr/ad-thumb/05fce9cdd7bd015d53b2c95e4cb3e3ca83010542.jpg', 'https://img5.leboncoin.fr/ad-thumb/1382fa897e2c25ed323fb4dfed87504b9fdacfae.jpg', 'https://img1.leboncoin.fr/ad-thumb/0366debe46a69eaf9ac9eee15f17150e5a49ec54.jpg', 'https://img2.leboncoin.fr/ad-thumb/7828308b93335be95aaf010598c736a74bb8851b.jpg', 'https://img1.leboncoin.fr/ad-thumb/1cb8a1b92b65f9fe5cc2eefd4d7d0daf8ec1cd98.jpg', 'https://img6.leboncoin.fr/ad-thumb/ae2b9eba587ce402ef374ac3b9868ff54bc5dd6c.jpg', 'https://img2.leboncoin.fr/ad-thumb/7c38a4cccd32f0a5537d79132e23acfc7eda2565.jpg', 'https://img3.leboncoin.fr/ad-thumb/67d555313224a8e88575eabadbe67e2f63ee35bc.jpg'], 'urls_large': ['https://img5.leboncoin.fr/ad-large/72dde7ad3585ed4ad2e98318bbb8db013a8c663a.jpg', 'https://img4.leboncoin.fr/ad-large/05fce9cdd7bd015d53b2c95e4cb3e3ca83010542.jpg', 'https://img5.leboncoin.fr/ad-large/1382fa897e2c25ed323fb4dfed87504b9fdacfae.jpg', 'https://img1.leboncoin.fr/ad-large/0366debe46a69eaf9ac9eee15f17150e5a49ec54.jpg', 'https://img2.leboncoin.fr/ad-large/7828308b93335be95aaf010598c736a74bb8851b.jpg', 'https://img1.leboncoin.fr/ad-large/1cb8a1b92b65f9fe5cc2eefd4d7d0daf8ec1cd98.jpg', 'https://img6.leboncoin.fr/ad-large/ae2b9eba587ce402ef374ac3b9868ff54bc5dd6c.jpg', 'https://img2.leboncoin.fr/ad-large/7c38a4cccd32f0a5537d79132e23acfc7eda2565.jpg', 'https://img3.leboncoin.fr/ad-large/67d555313224a8e88575eabadbe67e2f63ee35bc.jpg']}, 'url': 'https://www.leboncoin.fr/colocations/1684919633.htm', 'subject': 'Colocation Massy Opera', 'index_date': '2019-11-02 23:11:49', 'expiration_date': '2019-12-03 09:17:46', 'first_publication_date': '2019-10-04 09:17:46'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '77', 'department_name': 'Seine-et-Marne', 'city_label': 'Vaux-le-Pénil 77000', 'city': 'Vaux-le-Pénil', 'zipcode': '77000', 'lat': 48.52216, 'lng': 2.67608, 'source': 'address', 'provider': 'here', 'is_shape': False}, 'body': "Dans une résidence au calme de 4 appartements, au premier et dernier étage, F2 lumineux de 47 m², grand séjour avec cuisine à l'américaine meublée et équipée avec une hauteur sous plafond de 2,50 m, balcon, une grande chambre, salle de douche avec double vasque, wc séparé, dressing, fibre optique.\nChauffage par radiateur fluide caloporteur programmable, Electricité norme 2016. \nArrêt de bus à 200 m, gare de Melun et de Bois le roi à moins de 10 min.\nLoyer de 690 euros + 75 euros de provision de charges pour l'eau froide, l'eau chaude, l'entretien résidence et taxe d'ordure ménagère.\nLibre de suite.\nBail Notarié.", 'images': {'thumb_url': 'https://img0.leboncoin.fr/ad-thumb/0964ee41ed1068c519883408a565150dc8b4fcfd.jpg', 'small_url': 'https://img0.leboncoin.fr/ad-small/0964ee41ed1068c519883408a565150dc8b4fcfd.jpg', 'nb_images': 10, 'urls': ['https://img0.leboncoin.fr/ad-image/0964ee41ed1068c519883408a565150dc8b4fcfd.jpg', 'https://img4.leboncoin.fr/ad-image/43f7281aedc612434be8334ffcde447a3c64f6d3.jpg', 'https://img4.leboncoin.fr/ad-image/2b181f1a13ccd83350b58076d626f509c319b26a.jpg', 'https://img2.leboncoin.fr/ad-image/75d09f46fd4550ad6201095c2d629395c69af079.jpg', 'https://img0.leboncoin.fr/ad-image/6b5e5590b041f4b52b63a5f20bb9f0dc40213506.jpg', 'https://img4.leboncoin.fr/ad-image/031d53c97ef59ed663993cbecc8a0220ca3a131a.jpg', 'https://img3.leboncoin.fr/ad-image/60df84dfe1eb6afc6be6fcf8031e23b2a7db470e.jpg', 'https://img6.leboncoin.fr/ad-image/83e6a3f9b9103850af5815ed79e2ae88c1062d83.jpg', 'https://img1.leboncoin.fr/ad-image/b3bb5d2a8ea70021e1e5d82786bbd58626fa111d.jpg', 'https://img1.leboncoin.fr/ad-image/ae7b37036c3b674b0e509532c7ead618239913e3.jpg'], 'urls_thumb': ['https://img0.leboncoin.fr/ad-thumb/0964ee41ed1068c519883408a565150dc8b4fcfd.jpg', 'https://img4.leboncoin.fr/ad-thumb/43f7281aedc612434be8334ffcde447a3c64f6d3.jpg', 'https://img4.leboncoin.fr/ad-thumb/2b181f1a13ccd83350b58076d626f509c319b26a.jpg', 'https://img2.leboncoin.fr/ad-thumb/75d09f46fd4550ad6201095c2d629395c69af079.jpg', 'https://img0.leboncoin.fr/ad-thumb/6b5e5590b041f4b52b63a5f20bb9f0dc40213506.jpg', 'https://img4.leboncoin.fr/ad-thumb/031d53c97ef59ed663993cbecc8a0220ca3a131a.jpg', 'https://img3.leboncoin.fr/ad-thumb/60df84dfe1eb6afc6be6fcf8031e23b2a7db470e.jpg', 'https://img6.leboncoin.fr/ad-thumb/83e6a3f9b9103850af5815ed79e2ae88c1062d83.jpg', 'https://img1.leboncoin.fr/ad-thumb/b3bb5d2a8ea70021e1e5d82786bbd58626fa111d.jpg', 'https://img1.leboncoin.fr/ad-thumb/ae7b37036c3b674b0e509532c7ead618239913e3.jpg'], 'urls_large': ['https://img0.leboncoin.fr/ad-large/0964ee41ed1068c519883408a565150dc8b4fcfd.jpg', 'https://img4.leboncoin.fr/ad-large/43f7281aedc612434be8334ffcde447a3c64f6d3.jpg', 'https://img4.leboncoin.fr/ad-large/2b181f1a13ccd83350b58076d626f509c319b26a.jpg', 'https://img2.leboncoin.fr/ad-large/75d09f46fd4550ad6201095c2d629395c69af079.jpg', 'https://img0.leboncoin.fr/ad-large/6b5e5590b041f4b52b63a5f20bb9f0dc40213506.jpg', 'https://img4.leboncoin.fr/ad-large/031d53c97ef59ed663993cbecc8a0220ca3a131a.jpg', 'https://img3.leboncoin.fr/ad-large/60df84dfe1eb6afc6be6fcf8031e23b2a7db470e.jpg', 'https://img6.leboncoin.fr/ad-large/83e6a3f9b9103850af5815ed79e2ae88c1062d83.jpg', 'https://img1.leboncoin.fr/ad-large/b3bb5d2a8ea70021e1e5d82786bbd58626fa111d.jpg', 'https://img1.leboncoin.fr/ad-large/ae7b37036c3b674b0e509532c7ead618239913e3.jpg']}, 'url': 'https://www.leboncoin.fr/locations/1697044200.htm', 'subject': 'Grand F2 rénové à neuf de 47 m² avec balcon au calme', 'index_date': '2019-10-31 16:41:04', 'expiration_date': '2019-12-26 17:52:38', 'first_publication_date': '2019-10-27 17:52:38'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '75', 'department_name': 'Paris', 'city_label': 'Paris 75008', 'city': 'Paris', 'zipcode': '75008', 'lat': 48.88256, 'lng': 2.32369, 'source': 'address', 'provider': 'here', 'is_shape': False}, 'body': "A LOUER. SANS REPRISE. Emplacement n:1, Libre de suite 50 M2 Plein Pied , Tres belle façade, A cote de grande enseignes Aucun travaux a prévoir, Climatisation ,Grille extérieur, Pas de frais d'agence , Metro Clichy , Située au 29 boulevard des Batignolles paris 8", 'images': {'thumb_url': 'https://img4.leboncoin.fr/ad-thumb/443a9580f6a445b3edb223ca0152355045e498a3.jpg', 'small_url': 'https://img4.leboncoin.fr/ad-small/443a9580f6a445b3edb223ca0152355045e498a3.jpg', 'nb_images': 1, 'urls': ['https://img4.leboncoin.fr/ad-image/443a9580f6a445b3edb223ca0152355045e498a3.jpg'], 'urls_thumb': ['https://img4.leboncoin.fr/ad-thumb/443a9580f6a445b3edb223ca0152355045e498a3.jpg'], 'urls_large': ['https://img4.leboncoin.fr/ad-large/443a9580f6a445b3edb223ca0152355045e498a3.jpg']}, 'url': 'https://www.leboncoin.fr/bureaux_commerces/1654379819.htm', 'subject': 'Boutique A Louer', 'index_date': '2019-10-24 12:20:46', 'expiration_date': '2019-11-30 22:10:08', 'first_publication_date': '2019-10-01 22:10:08'}, {'location': {'region_id': '12', 'region_name': 'Ile-de-France', 'department_id': '91', 'department_name': 'Essonne', 'city_label': 'Brunoy 91800', 'city': 'Brunoy', 'zipcode': '91800', 'lat': 48.6982, 'lng': 2.50484, 'source': 'city', 'provider': 'lbc', 'is_shape': True}, 'body': "Appartement Brunoy 4 pièce(s) 85 m2 + DOUBLE BOX ET CAVE\n\nBEL APPARTEMENT F4 AU COEUR DU CENTRE VILLE et 2 MN DE LA GARE .Comprenant : Entrée, séjour double Donnant sur balcon avec vue dégagée cuisine Equipée, 2 chambres, salle de bains, salle d'eau, wc, rangements.. Garage double. Cave. RÉSIDENCE STANDING - RAVALEMENT 2018.. LE TOUT EN PARFAIT ETAT !\n419000 euros Honoraires à la charge du vendeur.\nRéférence annonce : E1GDT2\nLes honoraires sont à la charge du vendeur\n\nA propos de la copropriété :\nProcédure en cours\nNombre de lots : 74\nCharges prévisionnelles annuelles : 2592 €", 'images': {'thumb_url': 'https://img3.leboncoin.fr/ad-thumb/64dee61ad283b2ad7032b15ca2f18072859c32a7.jpg', 'small_url': 'https://img3.leboncoin.fr/ad-small/64dee61ad283b2ad7032b15ca2f18072859c32a7.jpg', 'nb_images': 9, 'urls': ['https://img3.leboncoin.fr/ad-image/64dee61ad283b2ad7032b15ca2f18072859c32a7.jpg', 'https://img6.leboncoin.fr/ad-image/d39afb0894610173d137f2560f76220827319051.jpg', 'https://img1.leboncoin.fr/ad-image/24702942b810c9e41672339d0818ac2e5e9066d0.jpg', 'https://img6.leboncoin.fr/ad-image/569dd3c1fbfc2a1315b23d04e70050812fe84afd.jpg', 'https://img3.leboncoin.fr/ad-image/fdcf8b5d2f08b872855c4a8da1bca4fbf06dd279.jpg', 'https://img2.leboncoin.fr/ad-image/7e9ec7b402d986c59f05aa4ec50b6d164e8d2f43.jpg', 'https://img6.leboncoin.fr/ad-image/f9706869cafe3a37a2fed83c05ff642adfff2474.jpg', 'https://img5.leboncoin.fr/ad-image/492d030fccac48ba3c7e85639e59162c6e1e65a0.jpg', 'https://img5.leboncoin.fr/ad-image/a6e3952e02891936bbaad48c54ad6ea696aad8a9.jpg'], 'urls_thumb': ['https://img3.leboncoin.fr/ad-thumb/64dee61ad283b2ad7032b15ca2f18072859c32a7.jpg', 'https://img6.leboncoin.fr/ad-thumb/d39afb0894610173d137f2560f76220827319051.jpg', 'https://img1.leboncoin.fr/ad-thumb/24702942b810c9e41672339d0818ac2e5e9066d0.jpg', 'https://img6.leboncoin.fr/ad-thumb/569dd3c1fbfc2a1315b23d04e70050812fe84afd.jpg', 'https://img3.leboncoin.fr/ad-thumb/fdcf8b5d2f08b872855c4a8da1bca4fbf06dd279.jpg', 'https://img2.leboncoin.fr/ad-thumb/7e9ec7b402d986c59f05aa4ec50b6d164e8d2f43.jpg', 'https://img6.leboncoin.fr/ad-thumb/f9706869cafe3a37a2fed83c05ff642adfff2474.jpg', 'https://img5.leboncoin.fr/ad-thumb/492d030fccac48ba3c7e85639e59162c6e1e65a0.jpg', 'https://img5.leboncoin.fr/ad-thumb/a6e3952e02891936bbaad48c54ad6ea696aad8a9.jpg'], 'urls_large': ['https://img3.leboncoin.fr/ad-large/64dee61ad283b2ad7032b15ca2f18072859c32a7.jpg', 'https://img6.leboncoin.fr/ad-large/d39afb0894610173d137f2560f76220827319051.jpg', 'https://img1.leboncoin.fr/ad-large/24702942b810c9e41672339d0818ac2e5e9066d0.jpg', 'https://img6.leboncoin.fr/ad-large/569dd3c1fbfc2a1315b23d04e70050812fe84afd.jpg', 'https://img3.leboncoin.fr/ad-large/fdcf8b5d2f08b872855c4a8da1bca4fbf06dd279.jpg', 'https://img2.leboncoin.fr/ad-large/7e9ec7b402d986c59f05aa4ec50b6d164e8d2f43.jpg', 'https://img6.leboncoin.fr/ad-large/f9706869cafe3a37a2fed83c05ff642adfff2474.jpg', 'https://img5.leboncoin.fr/ad-large/492d030fccac48ba3c7e85639e59162c6e1e65a0.jpg', 'https://img5.leboncoin.fr/ad-large/a6e3952e02891936bbaad48c54ad6ea696aad8a9.jpg']}, 'url': 'https://www.leboncoin.fr/ventes_immobilieres/1698801864.htm', 'subject': 'Appartement 4 pièces 85 m²', 'index_date': '2019-11-02 10:25:56', 'expiration_date': None, 'first_publication_date': '2019-10-30 07:18:47'}] 

In [33]:
pd.DataFrame(str_json)

ValueError: DataFrame constructor not properly called!